In [1]:
import sys
sys.path.append('../scripts')

import os
from utils import cleanCmpnyNm, getGrpCmpnyDict, convertCmpnyNm
import numpy as np
import pandas as pd

# 2019

In [2]:
year = 2019
dfCmpnySumry = pd.read_excel(f'../data/cmpnySumry/cmpnySumry{year}Kor.xlsx', thousands=',')
dfCmpnySumry.head()

,공개년월,회계년,기업집단명,소속회사명,대표자,설립일,계열편입일,업종코드,영위업종,종업원수,...,자산총액,부채총액,자본총액,자본금,매출액,당기순이익,구분,법인등록번호,year2,rcmgCode
0,201905,2018,DB,디비생명보험(주),이태운,1989/04/14,1997/01/03,K65,보험 및 연금업,563,...,10887144.0,10296289.0,590855.0,193331.0,2180825.0,24637.0,금융보험업,1101110623606,2019,821420.0
1,201905,2018,DB,디비금융서비스(주),황원기,2014/01/13,2014/03/01,K66,금융 및 보험관련 서비스업,30,...,6057.0,2460.0,3597.0,14001.0,13311.0,-2499.0,금융보험업,1101115314474,2019,23044292.0
2,201905,2018,DB,디비에프아이에스(주),강운식,2014/12/01,2018/06/01,J62,"컴퓨터 프로그래밍, 시스템 통합 및 관리업",217,...,47579.0,5854.0,41725.0,1000.0,47269.0,12063.0,일반회사,1101115576727,2019,24038019.0
3,201905,2018,DB,디비금융투자(주),고원종,1982/12/20,1982/12/20,K66,금융 및 보험관련 서비스업,892,...,5428689.0,4794353.0,634336.0,212232.0,725156.0,46854.0,금융보험업,1101110339956,2019,821640.0
4,201905,2018,DB,디비인베스트(주),최경진,2009/11/16,2010/01/04,M71,전문 서비스업,1,...,14912.0,17943.0,-3031.0,306500.0,933.0,-267.0,일반회사,1101114220359,2019,16003715.0


In [3]:
dfCmpnySumry['매출액Val'] = dfCmpnySumry['매출액']
dfCmpnySumry['매출회사id'] = dfCmpnySumry['rcmgCode']
dfCmpnySumry['매입회사id'] = dfCmpnySumry['rcmgCode']
dfCmpnySumry['매출사공개여부'] = ~dfCmpnySumry['기업공개일'].isna()
dfCmpnySumry['매출사상/비'] = dfCmpnySumry['매출사공개여부'].apply(lambda b: '상장' if b else '비상장')
dfCmpnySumry['매입사상/비'] = dfCmpnySumry['매출사상/비']
dfCmpnySumry['매출사금융여부'] = dfCmpnySumry['업종코드'].isin(['K64', 'K65', 'K66'])
dfCmpnySumry['매출사금융/비금융'] = dfCmpnySumry['매출사금융여부'].apply(lambda b: '금융' if b else '비금융')
dfCmpnySumry['매입사금융/비금융'] = dfCmpnySumry['매출사금융/비금융']

In [4]:
dfCmpnySumry['매출회사'] = cleanCmpnyNm(dfCmpnySumry['소속회사명'])
dfCmpnySumry['매입회사'] = cleanCmpnyNm(dfCmpnySumry['소속회사명'])

In [5]:
dfCmpnySumry[dfCmpnySumry['rcmgCode'].isna()]

,공개년월,회계년,기업집단명,소속회사명,대표자,설립일,계열편입일,업종코드,영위업종,종업원수,...,매출회사id,매입회사id,매출사공개여부,매출사상/비,매입사상/비,매출사금융여부,매출사금융/비금융,매입사금융/비금융,매출회사,매입회사
1962,201905,2018,네이버,(주)락스퀘어,조성환,2017/04/28,2018/03/01,J58,출판업,30,...,NaN,NaN,False,비상장,비상장,False,비금융,비금융,락스퀘어,락스퀘어


In [6]:
dfCmpnySumry.head()

,공개년월,회계년,기업집단명,소속회사명,대표자,설립일,계열편입일,업종코드,영위업종,종업원수,...,매출회사id,매입회사id,매출사공개여부,매출사상/비,매입사상/비,매출사금융여부,매출사금융/비금융,매입사금융/비금융,매출회사,매입회사
0,201905,2018,DB,디비생명보험(주),이태운,1989/04/14,1997/01/03,K65,보험 및 연금업,563,...,821420.0,821420.0,False,비상장,비상장,True,금융,금융,디비생명보험,디비생명보험
1,201905,2018,DB,디비금융서비스(주),황원기,2014/01/13,2014/03/01,K66,금융 및 보험관련 서비스업,30,...,23044292.0,23044292.0,False,비상장,비상장,True,금융,금융,디비금융서비스,디비금융서비스
2,201905,2018,DB,디비에프아이에스(주),강운식,2014/12/01,2018/06/01,J62,"컴퓨터 프로그래밍, 시스템 통합 및 관리업",217,...,24038019.0,24038019.0,False,비상장,비상장,False,비금융,비금융,디비에프아이에스,디비에프아이에스
3,201905,2018,DB,디비금융투자(주),고원종,1982/12/20,1982/12/20,K66,금융 및 보험관련 서비스업,892,...,821640.0,821640.0,True,상장,상장,True,금융,금융,디비금융투자,디비금융투자
4,201905,2018,DB,디비인베스트(주),최경진,2009/11/16,2010/01/04,M71,전문 서비스업,1,...,16003715.0,16003715.0,False,비상장,비상장,False,비금융,비금융,디비인베스트,디비인베스트


In [7]:
dfAppnGroupSttus = pd.read_excel(f"../data/grpSumry/appnGroupSttus/appnGroupSttus{year}.xlsx")
dfAppnGroupSttus = dfAppnGroupSttus[dfAppnGroupSttus['smerNm'].str.len() <= 3]
dfAppnGroupSttus = dfAppnGroupSttus.reset_index().drop(columns=['index'])
dfAppnGroupSttus['repreCmpny'] = cleanCmpnyNm(dfAppnGroupSttus['repreCmpny'])
dfAppnGroupSttus

,unityGrupNm,unityGrupCode,smerNm,repreCmpny,sumCmpnyCo,invstmntLmtt,entrprsCl
0,삼성,K1000032,이재용,삼성전자,62,상호출자제한집단,일반
1,현대자동차,K3000027,정몽구,현대자동차,53,상호출자제한집단,일반
2,에스케이,K1000050,최태원,에스케이,111,상호출자제한집단,일반
3,엘지,K1000051,구광모,엘지,75,상호출자제한집단,일반
4,롯데,K1000027,신동빈,롯데지주,95,상호출자제한집단,일반
5,한화,K1000082,김승연,한화,75,상호출자제한집단,일반
6,지에스,K3000076,허창수,지에스,64,상호출자제한집단,일반
7,현대중공업,K3000052,정몽준,한국조선해양,31,상호출자제한집단,일반
8,신세계,K1000042,이명희,신세계,40,상호출자제한집단,일반
9,한진,K1000081,조원태,한진칼,32,상호출자제한집단,일반


In [12]:
lookUpListBuy = []
lookUpListSell = []

for grp in dfAppnGroupSttus['unityGrupNm']:
    print("="*15, grp, "="*15)
    dfTrans = pd.read_excel(f"../data/transactions/{year}/{grp}.xlsx")
    dfTrans['매출액총계'] = dfTrans['국내매출액'] + dfTrans['해외매출액']
    dfTrans['year2'] = year
    dfTrans['year'] = year - 1
    dfTrans['grpname2'] = grp
    
    
    if grp in ['엘에스', '미래에셋', '태영', 'DB', 'KG']:
        dfTrans.loc[:, ['매출회사', '매입회사']] = convertCmpnyNm(dfTrans[['매출회사', '매입회사']], grp)
    
    dfTransId = dfTrans.merge(dfCmpnySumry[['매출회사', '매출회사id', '매출사상/비', '매출사금융/비금융', '매출액Val']], on='매출회사', how='left')
    dfTransId = dfTransId.merge(dfCmpnySumry[['매입회사', '매입회사id', '매입사상/비', '매입사금융/비금융']], on='매입회사', how='left')
    print(list(dfTransId[dfTransId['매출회사id'].isna()]['매출회사'].unique()))
    lookUpListSell.extend(list(dfTransId[dfTransId['매출회사id'].isna()]['매출회사'].unique()))
    print(list(dfTransId[dfTransId['매입회사id'].isna()]['매입회사'].unique()))
    lookUpListBuy.extend(list(dfTransId[dfTransId['매입회사id'].isna()]['매출회사'].unique()))
    print(dfTransId.head())
    
    if not os.path.exists(f'../data/transactions-mrg/{year}'):
        os.makedirs(f'../data/transactions-mrg/{year}')
    
    dfTransId.to_excel(f"../data/transactions-mrg/{year}/{grp}.xlsx", index=False)
    
    print("="*15, grp, "="*15)

=============== 삼성 ===============
[]
['에스프린팅솔루션']
   매출회사      매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액        해외매출액  \
0  삼성전자      삼성물산   78676      2269950  149436938.0  16821275  153560595.0   
1  삼성전자  삼성바이오에피스       1      2269950  149436938.0  16821275  153560595.0   
2  삼성전자   삼성디스플레이  164404      2269950  149436938.0  16821275  153560595.0   
3  삼성전자       세메스       5      2269950  149436938.0  16821275  153560595.0   
4  삼성전자   삼성전자서비스  456109      2269950  149436938.0  16821275  153560595.0   

         매출액총계  year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융       매출액Val  \
0  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   
1  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   
2  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   
3  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   
4  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   


=============== 엘지 ===============
=============== 롯데 ===============
[]
['롯데디에프글로벌', '롯데닷컴', '롯데백화점마산', '롯데시네마', '롯데로지스틱스', '롯데아이티테크']
   매출회사      매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액  해외매출액   매출액총계  \
0  롯데지주      호텔롯데  12284       167669          NaN  255644   7296  262940   
1  롯데지주   롯데면세점제주   1140       167669          NaN  255644   7296  262940   
2  롯데지주    부산롯데호텔   1073       167669          NaN  255644   7296  262940   
3  롯데지주      롯데물산    913       167669          NaN  255644   7296  262940   
4  롯데지주  롯데디에프글로벌      1       167669          NaN  255644   7296  262940   

   year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융    매출액Val        매입회사id  \
0   2019  2018       롯데  310204.0     상장       비금융  262940.0  6.300200e+05   
1   2019  2018       롯데  310204.0     상장       비금융  262940.0  1.801708e+09   
2   2019  2018       롯데  310204.0     상장       비금융  262940.0  6.306830e+05   
3   2019  2018       롯데  310204.0     상장       비금융  262940.0  6.208900e+05   
4   2019  2

=============== 신세계 ===============
=============== 한진 ===============
[]
['한국글로발로지스틱시스템', 'HANJININTERMODALAMERICAINC.', 'HANJINGLOBALLOGISTICS(VIETNAM)Ltd.', "HanjinInt'lCorp.", "QINGDAOHANJINLUHAIINT'LLOGISTICSCO.,LTD", 'EURASIALOGISTICSSERVICELLC', 'HanjinGlobalLogistics(HongKong)Ltd.', 'HANJINGLOBALLOGISTICS(SHANGHAI)LIMITED', 'HanjinGlobalLogistics(Dalian)Ltd.', 'HANJINGLOBALLOGISTICS(SHENZHEN)LIMITED', 'HANJINGLOBALLOGISTICSGUANGZHOUCO.,LTD', 'HANJINGLOBALLOGISTICSEUROPEs.r.o.', 'GLSAP', "WACINT'LLOGISTICS(HKLIMITED)", "WACINT'LLOGISTICS(QINGDAOCOMPANYLIMITED)"]
   매출회사  매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액       해외매출액  \
0  대한항공   한진칼   178       199900       1714.0  1183532  11463333.0   
1  대한항공    한진  3347       199900       1714.0  1183532  11463333.0   
2  대한항공  한국공항  1003       199900       1714.0  1183532  11463333.0   
3  대한항공  정석기업     1       199900       1714.0  1183532  11463333.0   
4  대한항공  한진관광   112       199900       1714.0  1183532  11463333.0   

   

    매출회사       매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액  해외매출액  \
0  현대백화점       한무쇼핑  95935.0       137411          NaN  1376064      0   
1  현대백화점   현대백화점면세점    566.0       137411          NaN  1376064      0   
2  현대백화점     현대그린푸드  19039.0       137411          NaN  1376064      0   
3  현대백화점   현대캐터링시스템     36.0       137411          NaN  1376064      0   
4  현대백화점  씨엔에스푸드시스템     12.0       137411          NaN  1376064      0   

     매출액총계  year2  year grpname2   매출회사id 매출사상/비 매출사금융/비금융     매출액Val  \
0  1376064   2019  2018    현대백화점  86512.0     상장       비금융  1376064.0   
1  1376064   2019  2018    현대백화점  86512.0     상장       비금융  1376064.0   
2  1376064   2019  2018    현대백화점  86512.0     상장       비금융  1376064.0   
3  1376064   2019  2018    현대백화점  86512.0     상장       비금융  1376064.0   
4  1376064   2019  2018    현대백화점  86512.0     상장       비금융  1376064.0   

         매입회사id 매입사상/비 매입사금융/비금융  
0  6.123910e+05    비상장       비금융  
1  1.803206e+09    비상장       비금융  
2  6.201140

[]
['거래계열회사수', '피오르드프로세싱코리아']
  매출회사      매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)  국내매출액  해외매출액    매출액총계  \
0  코오롱  코오롱인더스트리  19150        43413        169.0  58004  169.0  58173.0   
1  코오롱   코오롱플라스틱   2213        43413        169.0  58004  169.0  58173.0   
2  코오롱   코오롱머티리얼   2440        43413        169.0  58004  169.0  58173.0   
3  코오롱    코오롱글로텍   3859        43413        169.0  58004  169.0  58173.0   
4  코오롱      스위트밀     82        43413        169.0  58004  169.0  58173.0   

   year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융   매출액Val      매입회사id  \
0   2019  2018      코오롱  320234.0     상장       비금융  58173.0  16003302.0   
1   2019  2018      코오롱  320234.0     상장       비금융  58173.0    825841.0   
2   2019  2018      코오롱  320234.0     상장       비금융  58173.0  13004456.0   
3   2019  2018      코오롱  320234.0     상장       비금융  58173.0    325732.0   
4   2019  2018      코오롱  320234.0     상장       비금융  58173.0    389116.0   

  매입사상/비 매입사금융/비금융  
0     상장       비금융  
1     상장       비금융  

['한국테크놀로지그룹', '한국타이어앤테크놀로지', '한국아트라스비엑스', '한국카앤라이프', '한국네트웍스', '한국엔지니어링웍스', '한국프리시전웍스']
['한국타이어앤테크놀로지', '한국카앤라이프', '한국네트웍스', '한국테크놀로지그룹', '한국아트라스비엑스', '한국엔지니어링웍스', '한국프리시전웍스']
          매출회사         매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액  \
0    한국테크놀로지그룹  한국타이어앤테크놀로지  57504        57553      57553.0   74874   
1    한국테크놀로지그룹      한국카앤라이프     35        57553      57553.0   74874   
2    한국테크놀로지그룹       한국네트웍스      8        57553      57553.0   74874   
3    한국테크놀로지그룹       신양월드레저      6        57553      57553.0   74874   
4  한국타이어앤테크놀로지    한국테크놀로지그룹   1658         2844    2154809.0  915864   

       해외매출액      매출액총계  year2  year grpname2  매출회사id 매출사상/비 매출사금융/비금융  \
0        NaN        NaN   2019  2018    한국타이어     NaN    NaN       NaN   
1        NaN        NaN   2019  2018    한국타이어     NaN    NaN       NaN   
2        NaN        NaN   2019  2018    한국타이어     NaN    NaN       NaN   
3        NaN        NaN   2019  2018    한국타이어     NaN    NaN       NaN   
4  2277365.0  3193229

=============== 네이버 ===============
=============== 태영 ===============
[]
['에스비에스에이엔티', '에스비에스internationalInc', '케이씨피', 'PLANKEntertainment']
   매출회사      매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액   해외매출액  \
0  태영건설   태영인더스트리     148       850294          NaN  1991815  4234.0   
1  태영건설      유니시티  476421       850294          NaN  1991815  4234.0   
2  태영건설    에코시티개발  133049       850294          NaN  1991815  4234.0   
3  태영건설    엠시에타개발  207263       850294          NaN  1991815  4234.0   
4  태영건설  포천바이오에너지    1049       850294          NaN  1991815  4234.0   

       매출액총계  year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융     매출액Val  \
0  1996049.0   2019  2018       태영  501662.0     상장       비금융  1996049.0   
1  1996049.0   2019  2018       태영  501662.0     상장       비금융  1996049.0   
2  1996049.0   2019  2018       태영  501662.0     상장       비금융  1996049.0   
3  1996049.0   2019  2018       태영  501662.0     상장       비금융  1996049.0   
4  1996049.0   2019  2018       태영  501662.0

=============== 유진 ===============
=============== 금호석유화학 ===============
[]
['상해금호일려소료유한공사', '광동금호일려고분자재료유한공사', '남경금포금호화공유한공사', '강소금호양농화공유한공사']
     매출회사     매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액      해외매출액  \
0  금호석유화학  금호피앤비화학  161794       199963      22370.0  1336599  2562974.0   
1  금호석유화학  금호미쓰이화학    7583       199963      22370.0  1336599  2562974.0   
2  금호석유화학  금호미쓰이화학    7583       199963      22370.0  1336599  2562974.0   
3  금호석유화학  금호미쓰이화학    7583       199963      22370.0  1336599  2562974.0   
4  금호석유화학  금호미쓰이화학    7583       199963      22370.0  1336599  2562974.0   

       매출액총계  year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융     매출액Val  \
0  3899573.0   2019  2018   금호석유화학  351350.0     상장       비금융  3899573.0   
1  3899573.0   2019  2018   금호석유화학  351350.0     상장       비금융  3899573.0   
2  3899573.0   2019  2018   금호석유화학  351350.0     상장       비금융  3899573.0   
3  3899573.0   2019  2018   금호석유화학  351350.0     상장       비금융  3899573.0   
4  3899573.0   201

In [13]:
len(lookUpListSell)

14

In [15]:
lookUpListSell

['현대에너지솔루션',
 '돈돈팜',
 '미래에셋파트너스제구호사모투자',
 '싱그린FS',
 '농업회사법인팜앤코',
 '한국테크놀로지그룹',
 '한국타이어앤테크놀로지',
 '한국아트라스비엑스',
 '한국카앤라이프',
 '한국네트웍스',
 '한국엔지니어링웍스',
 '한국프리시전웍스',
 '제이비당진페럼빌전문투자형사모투자',
 '유진에버베스트턴어라운드기업재무안정사모투자']

In [14]:
len(lookUpListBuy)

573

In [17]:
# merge cmpnySumry
lCmpnySumry = []
for year in range(2020, 2022):
    df = pd.read_excel(f'../data/cmpnySumry/cmpnySumry{year}Kor.xlsx')
    lCmpnySumry.append(df)
    print(len(list(df.columns)))
dfCmpnySumry = pd.concat(lCmpnySumry, axis=0)
dfCmpnySumry.head()

23
23


,공개년월,회계년,기업집단명,소속회사명,대표자,설립일,계열편입일,업종코드,영위업종,종업원수,...,자산총액,부채총액,자본총액,자본금,매출액,당기순이익,구분,법인등록번호,year2,rcmgCode
0,202005,2019,넥슨,(유)가승개발,서춘식,2016/01/21,2016/01/21,R91,스포츠 및 오락관련 서비스업,3,...,"15,208","6,064","9,144",100,0,-465,일반회사,1345140003989,2020,1.802408e+09
1,202005,2019,지에스,(유)가승개발,서춘식,2016/01/21,2016/04/01,R91,스포츠 및 오락관련 서비스업,3,...,"15,208","6,064","9,144",100,NaN,-465,일반회사,1345140003989,2020,1.802408e+09
2,202005,2019,카카오,(유)매디슨카운티의다리,이명한,2016/11/16,2020/02/03,R90,"창작, 예술 및 여가관련 서비스업",0,...,11,28,-17,10,3,2,일반회사,1101140192605,2020,1.803409e+09
3,202005,2019,엘에스,(유)빌드윈문경태양광발전,김선철,2018/12/11,2019/02/01,D35,"전기, 가스, 증기 및 공기 조절 공급업",1,...,331,0,331,331,0,0,일반회사,1754140000582,2020,2.102105e+09
4,202005,2019,엘에스,(유)빌드윈인동태양광발전,김선철,2018/12/11,2019/02/01,D35,"전기, 가스, 증기 및 공기 조절 공급업",1,...,"4,883","4,140",743,756,0,-13,일반회사,1760140002014,2020,2.102105e+09


In [18]:
dfCmpnySumry.shape

(4896, 23)

In [19]:
dfCmpnySumry.columns

Index(['공개년월', '회계년', '기업집단명', '소속회사명', '대표자', '설립일', '계열편입일', '업종코드', '영위업종',
       '종업원수', '결산기일', '결산주총일', '기업공개일', '자산총액', '부채총액', '자본총액', '자본금', '매출액',
       '당기순이익', '구분', '법인등록번호', 'year2', 'rcmgCode'],
      dtype='object')

In [20]:
len(dfCmpnySumry.columns)

23

In [22]:
dfCmpnySumry.columns = ['publicYm', 'fisclYr', 'unityGrupNm', 'entrprsNm', 'rprsntvNm', 'fondDe',\
                          'grinil', 'indutyCode', 'indutyNm', 'ordtmEmplyCo', 'stacntDudt', 'beforeBsnsStacntDe',\
                          'entrprsOthbcDe', 'assetsTotamt', 'debtTotamt', 'caplTotamt', 'caplAmount', 'selngAmount', 'thstrmNtpfAmount',\
                          'reconstrctur', 'jurirno', 'year2', 'rcmgCode']

In [23]:
dfCmpnySumry['entrprsNm'] = cleanCmpnyNm(dfCmpnySumry['entrprsNm'])
dfCmpnySumry['entrprsNm'].nunique()

2833

In [8]:
dfCmpnySumry.to_excel(f'../cmpnySumry2001-2021Eng.xlsx')

In [10]:
dfCmpnySumry.columns

Index(['publicYm', 'fisclYr', 'unityGrupNm', 'entrprsNm', 'rprsntvNm',
       'fondDe', 'grinil', 'indutyCode', 'indutyNm', 'ordtmEmplyCo',
       'stacntDudt', 'beforeBsnsStacntDe', 'entrprsOthbcDe', 'assetsTotamt',
       'debtTotamt', 'caplTotamt', 'caplAmount', 'selngAmount',
       'thstrmNtpfAmount', 'reconstrctur', 'jurirno', 'year2', 'rcmgCode'],
      dtype='object')

In [ ]:
dfCmpnySumry.columns = ['publicYm', 'fisclYr', 'unityGrupNm', 'entrprsNm', 'rprsntvNm', 'fondDe',\
                        'grinil', 'indutyCode', 'indutyNm', 'ordtmEmplyCo', 'stacntDudt', 'beforeBsnsStacntDe',\
                        'entrprsOthbcDe', 'assetsTotamt', 'debtTotamt', 'caplTotamt', 'caplAmount', 'selngAmount', 'thstrmNtpfAmount',\
                        'reconstrctur', 'jurirno', 'year2']

# 2019

In [1]:
import sys
sys.path.append('../scripts')

import api

In [2]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
myApi = api.Api(apiKey)
myApi.getTransTableAll(2019)

===============삼성===============
No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['삼성전자' '삼성물산' '서울레이크사이드' '삼우종합건축사사무소' '씨브이네트' '삼성바이오로직스' '삼성디스플레이'
 '삼성코닝어드밴스드글라스' '에스유머티리얼스' '스테코' '세메스' '삼성전자서비스' '삼성전자서비스씨에스' '삼성전자판매'
 '삼성전자로지텍' '수원삼성축' '삼성메디슨' '삼성에스디아이' '삼성전기' '삼성중공업' '에스디플렉스' '제일패션리테일'
 '삼성웰스토리' '시큐아이' '멀티캠퍼스' '에스티엠' '삼성에스디에스' '에스코어' '오픈핸즈' '미라콤아이앤씨'
 '삼성엔지니어링' '에스원' '휴먼티에스에스' '에스원씨알엠' '제일기획' '호텔신라' '신라스테이' '에이치디씨신라면세점'
 '에스비티엠' '삼성경제연' '삼성라이온즈' '삼성화재해상보험' '삼성화재애니카손해사정' '삼성화재서비스손해사정'
 '삼성화재금융서비스보험대리점' '삼성생명보험' '삼성선물' '삼성자산운용' '생보부동산신탁' '삼성액티브자산운용'
 '삼성생명서비스손해사정' '삼성에스알에이자산운용' '삼성생명금융서비스보험대리점' '삼성증권' '삼성카드' '삼성카드고객서비스']
['삼성물산' '삼성바이오에피스' '삼성디스플레이' '세메스' '삼성전자서비스' '삼성전자판매' '삼성메디슨' '삼성에스디아이'
 '삼성전기' '삼성웰스토리' '시큐아이' '삼성에스디에스' '삼성엔지니어링' '에스원' '제일기획' '호텔신라' '삼성화재해상보험'
 '삼성생명보험' '삼성자산운용' '삼성증권' '삼성카드' '삼성전자' '서울레이크사이드' '삼우종합건축사사무소' '씨브이네트'
 '삼성바이오로직스' '삼성코닝어드밴스드글라스' '에스유머티리얼스' '스테코' '삼성전자로지텍' '수원삼

['엘지' '엘지화학' '씨텍' '행복누리' '팜한농' '엘지하우시스' '엘지토스템비엠' '그린누리' '엘지생활건강' '코카콜라음료'
 '한국음료' '해태에이치티비' '더페이스샵' '크린소울' '씨앤피코스메틱스' '캐이엔아이' '에프엠지' '엘지파루크' '밝은누리'
 '태극제약' '제이에스제약' '엘지엠엠에이' '엘지전자' '엘지디스플레이' '나눔누리' '엘지이노텍' '이노위드' '하이프라자'
 '하이엠솔루텍' '하이텔레서비스' '에이스냉동공조' '하이엔텍' '엘지히타치워터솔루션' '하누리' '로보스타' '실리콘웍스'
 '엘지유플러스' '씨에스리더' '아인텔레서비스' '미디어로그' '데이콤크로싱' '씨에스원파트너' '위드유' '엘지씨엔에스'
 '비즈테크파트너스' '행복마루' '에스앤아이코퍼레이션' '곤지암예원' '미래엠' '드림누리' '서브원' '엘지경영개발원'
 '엘지스포츠' '지투알' '에이치에스애드' '엘베스트' '엘지상사' '판토스' '판토스부산신항물류센터' '헬리스타항공' '한울타리']
['엘지화학' '엘지하우시스' '엘지토스템비엠' '엘지생활건강' '더페이스샵' '엘지파루크' '엘지엠엠에이' '엘지전자'
 '엘지디스플레이' '엘지이노텍' '하이프라자' '엘지히타치워터솔루션' '엘지유플러스' '엘지씨엔에스' '에스앤아이코퍼레이션'
 '엘지경영개발원' '엘지스포츠' '에이치에스애드' '엘지상사' '판토스' '엘지엔시스' 'LGHoldingsJapanCo.,Ltd'
 '엘지' '씨텍' '행복누리' '팜한농' '코카콜라음료' '하이엔텍' '서브원' '팜화옹' '곤지암예원' '하우시스이엔지'
 '한국음료' '해태에이치티비' '크린소울' '씨앤피코스메틱스' '캐이엔아이' '에프엠지' '밝은누리' '태극제약' '하이엠솔루텍'
 '씨에스리더' '비즈테크파트너스' '미래엠' '제이에스제약' '하이텔레서비스' '에이스냉동공조' '하누리'
 '엘지퓨얼셀시스템즈코리아' '실리콘웍스' '아인텔레서비스' '미디어로그' '씨에스원파트너' '위드유' '지투알' '엘베스트'
 '

['현대중공업' '현대미포조선' '현대삼호중공업' '현대오일뱅크' '현대이엔티' '바르질라현대엔진' '현대중공업스포츠' '현대힘스'
 '코마스' '현대코스모' '현대오일터미널' '현대쉘베이스오일' '현대글로벌서비스' '현대케미칼' '현대오씨아이'
 '현대중공업터보기계' '현대중공업모스' '현대에너지솔루션' '현대중공업지주' '현대건설기계' '현대일렉트릭앤에너지시스템'
 '현대중공업파워시스템' '티앤에이']
['현대미포조선' '현대삼호중공업' '현대오일뱅크' '현대이엔티' '현대중공업스포츠' '현대힘스' '코마스' '현대오일터미널'
 '현대쉘베이스오일' '현대글로벌서비스' '현대케미칼' '현대오씨아이' '현대중공업터보기계' '현대중공업모스' '현대에너지솔루션'
 '현대중공업지주' '현대건설기계' '현대일렉트릭앤에너지시스템' '현대중공업파워시스템' '하이투자증권' '하이자산운용'
 '현대중공(중국)투자유한공사' '북경현대경성공정기계유한공사' '연대현대빙윤중공유한공사'
 'HyundaiVinashinShipyard' 'HyundaiArabiaLLC' '현대중공(중국)전기유한공사'
 '상주현대액압기기유한공사' '현대강소공정기계유한공사' '현대중공업' '바르질라현대엔진' '현대코스모'
 'HyundaiOilSingaporePte.Ltd' 'HyundaiOilbank(Shanghai)Co.,Ltd'
 'HyundaiGlobalServiceEuropeB.V' 'HyundaiGlobalServiceAmericasCo.,Ltd'
 'HyundaiGlobalServiceSingaporePte.Ltd' 'HyundaiPowerTransformersUSAInc'
 '현대(산동)중공업기계유한공사'
 'HyundaiHeavyIndustriesBrasil-Manufac-turingandTradingofConstructionEquipment'
 'PT.HyundaiConstruc-tionEquipmentAsia'
 'HyundaiConstruc-tionEquipmentIndiaPrivat

['두산' '두산중공업' '두산인프라코어' '두산밥캣' '두산건설' '오리콤' '두산베어스' '두산큐벡스' '두산밥캣코리아'
 '두산로보틱스' '디엘아이' '한컴' '밸류웍스' '오성파워오엔엠' '네오플럭스']
['두산중공업' '두산인프라코어' '두산밥캣' '두산건설' '오리콤' '두산베어스' '두산큐벡스' '네오트랜스' '두산밥캣코리아'
 '두산로보틱스' '두산모빌리티이노베이션' '디엘아이' '한컴' '두산메카텍' '오성파워오엔엠' '두산엔진' '디아이피홀딩스'
 '두타몰' '네오플럭스' 'DoosanMottrolJiangyinCo.,Ltd'
 'DoosanIndustrialVehicleAmericaCorp' 'DoosanFuelCellAmericaInc'
 'DoosanIndustrialVehicleEuropeN.A' 'DoosanIndustrialVehicleYantaiCo.,LTD'
 'DoosanIndustrialVehicleU.K.Ltd' 'DoosanElectro-MaterialsSingaporePteLtd'
 'DoosanElectro-MaterialsLuxembourgSarl'
 'DoosanElectro-Materials(Changshu)Co.,Ltd'
 'DoosanShanghaiChemicalMaterialsCo.,Ltd' 'CircuitFoilLuxembourgsarl'
 'DoosanHongkongLtd' 'DoosanDigitalInnovationAmerica,LLC'
 'DoosanDigitalInnovationChina' 'ClarkEquipmentCo'
 'DoosanElectro-Materials(ShenZhen)Limited'
 'DoosanDigitalInnovationEuropeLtd' 'DoosanBobcatEMEAs.r.o'
 'DoosanHeavyIndustriesVietnamCo.,Ltd' 'DoosanInfracoreEurope'
 'DoosanBobcatIndiaPrivateLtd' 'DoosanInfracor

['현대백화점' '한무쇼핑' '현대쇼핑' '현대백화점면세점' '현대그린푸드' '현대캐터링시스템' '씨엔에스푸드시스템' '현대홈쇼핑'
 '이에이치케이이앤에스' '현대에이치씨엔' '현대에이치씨엔서초케이블' '현대미디어' '에브리온티브이' '한섬' '현대지앤에프'
 '현대리바트' '현대리바트엠앤에스' '현대엘앤씨' '현대렌탈케어' '에버다임' '에버다임락툴' '타이포스' '한국타워크레인'
 '현대드림투어' '현대아이티앤이']
['한무쇼핑' '현대백화점면세점' '현대그린푸드' '현대캐터링시스템' '씨엔에스푸드시스템' '매그놀리아코리아' '현대홈쇼핑'
 '현대에이치씨엔' '현대에이치씨엔서초케이블' '현대미디어' '한섬' '현대지앤에프' '현대리바트' '현대엘앤씨' '현대렌탈케어'
 '에버다임' '현대드림투어' '현대아이티앤이' '한섬글로벌' '금강에이앤디' '현대백화점' '현대엘이디' '에이치케이이앤에스'
 '이이에치케이이앤에스' '동삼현대한매고분' '에브리온티브이' '현대리바트엠앤에스' '한섬파리' '한국타워크레인'
 'HYUNDAILIVARTVINA' 'LIVARTCANADA' 'HyundaiL&CCanadaInc'
 'HyundaiL&C(Shanghai)Co.,LTD' 'HyundaiL&CUSA,LLC' 'HyundaiL&CEuropeGmbH'
 'AmericanSurfacingMaterials,LLC' '에버다임락툴' '타이포스' '한우공정기계(상해)유한공사'
 'EVERDIGMHEMTLLC' 'EVERDIGMMONGOLIALLC' 'EVERDIGMAMERICAINC']
    매출회사       매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)      국내매출액  해외매출액
0  현대백화점       한무쇼핑  95935.0     137411.0          NaN  1376064.0    0.0
1  현대백화점   현대백화점면세점    566.0     137411.0          NaN  1376064.0    0.0
2  

['교보생명보험' '교보악사자산운용' '교보증권' '생보부동산신탁' '케이씨에이손해사정' '교보데이터센터' '교보리얼코'
 '교보정보통신' '교보핫트랙스' '제일안전서비스' '교보문고' '케이씨에이서비스']
['교보라이프플래닛생명보험' '교보악사자산운용' '교보증권' '생보부동산신탁' '케이씨에이손해사정' '교보리얼코' '교보문고'
 '교보정보통신' '교보핫트랙스' '제일안전서비스' '케이씨에이서비스' '교보생명보험' '교보생명자산운용-미국' '교보데이터센터']
     매출회사          매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)       국내매출액  해외매출액
0  교보생명보험  교보라이프플래닛생명보험     8.0      16135.0          NaN  12505561.0    NaN
1  교보생명보험      교보악사자산운용  1362.0      16135.0          NaN  12505561.0    NaN
2  교보생명보험          교보증권   478.0      16135.0          NaN  12505561.0    NaN
3  교보생명보험       생보부동산신탁    52.0      16135.0          NaN  12505561.0    NaN
4  교보생명보험     케이씨에이손해사정   591.0      16135.0          NaN  12505561.0    NaN
        매출회사          매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액  해외매출액
47      교보문고        교보생명보험   1728.0       3937.0          NaN  568448.0    NaN
48      교보문고  교보라이프플래닛생명보험     38.0       3937.0          NaN  568448.0    NaN
49      교보문고          교보증권      6.0     

['케이씨씨' '케이씨씨건설' '코리아오토글라스' '금강레저' '세우실업' '동주상사' '동주' '대호포장' '티앤케이정보'
 '주령금속']
['케이씨씨건설' '코리아오토글라스' '세우실업' 'KCS' 'KCK' 'KCB' 'KCG' 'KCQ' 'KCT' 'KCI'
 'KCID' 'KCV' 'KCC(Hanoi)' 'KCCPaint' 'Basildon' '케이씨씨' '동주' '대호포장']
   매출회사      매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)      국내매출액     해외매출액
0  케이씨씨    케이씨씨건설   26247.0     141289.0     127147.0  2776961.0  570375.0
1  케이씨씨  코리아오토글라스  115037.0     141289.0     127147.0  2776961.0  570375.0
2  케이씨씨      세우실업       5.0     141289.0     127147.0  2776961.0  570375.0
3  케이씨씨       KCS    4812.0     141289.0     127147.0  2776961.0  570375.0
4  케이씨씨       KCK   28373.0     141289.0     127147.0  2776961.0  570375.0
      매출회사      매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액  해외매출액
28    대호포장        동주    36.0         36.0          NaN   317.0    NaN
29  티앤케이정보      케이씨씨  3449.0       3786.0          NaN  3840.0    NaN
30  티앤케이정보    케이씨씨건설   282.0       3786.0          NaN  3840.0    NaN
31  티앤케이정보  코리아오토글라스    55.0       3786.0          N

['셀트리온홀딩스' '셀트리온' '셀트리온헬스케어' '셀트리온스킨큐어' '셀트리온제약' '티에스이엔씨' '티에스이엔엠']
['셀트리온엔터테인먼트' '셀트리온헬스케어' '셀트리온스킨큐어' '셀트리온' '셀트리온제약' '셀트리온화학연'
 '셀트리온충북대코스메틱스' '셀트리온충북대바이오메딕스' '티에스이엔엠']
       매출회사        매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액  \
0   셀트리온홀딩스  셀트리온엔터테인먼트     120.0        151.0          NaN     179.0   
1   셀트리온홀딩스    셀트리온헬스케어      31.0        151.0          NaN     179.0   
2      셀트리온    셀트리온스킨큐어     294.0     771739.0      15103.0  728307.0   
3      셀트리온    셀트리온헬스케어  771445.0     771739.0      15103.0  728307.0   
4  셀트리온헬스케어        셀트리온   21852.0      24200.0     626581.0   24200.0   

      해외매출액  
0       NaN  
1       NaN  
2  133577.0  
3  133577.0  
4  913103.0  
        매출회사           매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액  \
12  셀트리온스킨큐어  셀트리온충북대바이오메딕스      1.0        653.0        102.0   32135.0   
13    셀트리온제약           셀트리온  22613.0      22613.0          NaN  138504.0   
14    티에스이엔씨         셀트리온제약    184.0       2655.0          NaN    9659.0   
1

['엔엑스씨' '엔엑스씨엘' '소호브릭스' '와이즈키즈' '엔엑스프로퍼티스' '넥슨코리아' '넥슨네트웍스' '넥슨스페이스' '네오플'
 '넥슨지티' '띵소프트' '넥슨커뮤니케이션즈' '불리언게임즈' '넥슨레드' '엔미디어플랫폼' '중앙판교개발']
['엔엑스씨엘' '소호브릭스' '넥슨코리아' '네오플' '넥슨지티' '넥슨레드' '브이아이피사모주식형펀드일호' '엔엑스씨'
 '넥슨스페이스' '엔엑스프로퍼티스' '스토케코리아' '와이즈키즈' '엔엑스브이피' '넥슨네트웍스' '띵소프트' '넥슨커뮤니케이션즈'
 '불리언게임즈' '엔미디어플랫폼' '엔도어즈']
   매출회사   매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액   해외매출액
0  엔엑스씨  엔엑스씨엘    65.0      10279.0       5781.0  15787.0  6195.0
1  엔엑스씨  소호브릭스   409.0      10279.0       5781.0  15787.0  6195.0
2  엔엑스씨  넥슨코리아  2841.0      10279.0       5781.0  15787.0  6195.0
3  엔엑스씨    네오플  1333.0      10279.0       5781.0  15787.0  6195.0
4  엔엑스씨   넥슨지티     0.0      10279.0       5781.0  15787.0  6195.0
       매출회사   매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액  해외매출액
48     넥슨레드   넥슨지티  8581.0      11162.0          NaN  11459.0  496.0
49  엔미디어플랫폼  넥슨코리아  4939.0       5117.0          NaN  17065.0    NaN
50  엔미디어플랫폼    네오플    20.0       5117.0          NaN  17065.0    NaN
51  엔미디어플랫폼   넥슨지티

['금호석유화학' '금호피앤비화학' '금호미쓰이화학' '금호폴리켐' '금호티앤엘' '금호개발상사' '여수페트로']
['금호피앤비화학' '금호미쓰이화학' '금호폴리켐' '금호개발상사' '금호티앤엘' '코리아에너지발전소' '상해금호일려소료유한공사'
 '광동금호일려고분자재료유한공사' '남경금포금호화공유한공사' '금호석유화학' '강소금호양농화공유한공사']
     매출회사     매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)      국내매출액      해외매출액
0  금호석유화학  금호피앤비화학  161794.0     199963.0      22370.0  1336599.0  2562974.0
1  금호석유화학  금호미쓰이화학    7583.0     199963.0      22370.0  1336599.0  2562974.0
2  금호석유화학    금호폴리켐   28109.0     199963.0      22370.0  1336599.0  2562974.0
3  금호석유화학   금호개발상사     501.0     199963.0      22370.0  1336599.0  2562974.0
4  금호석유화학    금호티앤엘    1970.0     199963.0      22370.0  1336599.0  2562974.0
      매출회사     매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액   해외매출액
22  금호개발상사  금호미쓰이화학   166.0      11862.0          NaN  46471.0  1673.0
23  금호개발상사    금호폴리켐     3.0      11862.0          NaN  46471.0  1673.0
24  금호개발상사    금호티앤엘  1563.0      11862.0          NaN  46471.0  1673.0
25   여수페트로   금호석유화학    65.0        279.0          NaN   12

In [3]:
myApi.getTransTableAll(2020)

===============삼성===============
['삼성전자' '삼성물산' '서울레이크사이드' '삼우종합건축사사무소' '씨브이네트' '삼성바이오로직스' '삼성디스플레이'
 '삼성코닝어드밴스드글라스' '에스유머티리얼스' '스테코' '세메스' '삼성전자서비스' '삼성전자서비스씨에스' '삼성전자판매'
 '삼성전자로지텍' '수원삼성축' '삼성메디슨' '삼성에스디아이' '삼성전기' '삼성중공업' '에스디플렉스' '제일패션리테일'
 '삼성웰스토리' '시큐아이' '멀티캠퍼스' '에스티엠' '삼성에스디에스' '에스코어' '오픈핸즈' '미라콤아이앤씨'
 '삼성엔지니어링' '에스원' '휴먼티에스에스' '에스원씨알엠' '제일기획' '호텔신라' '신라스테이' '에이치디씨신라면세점'
 '에스비티엠' '삼성경제연' '삼성라이온즈' '삼성화재해상보험' '삼성화재애니카손해사정' '삼성화재서비스손해사정'
 '삼성화재금융서비스보험대리점' '삼성생명보험' '삼성선물' '삼성자산운용' '삼성액티브자산운용' '삼성생명서비스손해사정'
 '삼성에스알에이자산운용' '삼성생명금융서비스보험대리점' '삼성증권' '삼성카드' '삼성카드고객서비스']
['삼성물산' '삼성바이오에피스' '삼성디스플레이' '세메스' '삼성전자서비스' '삼성전자판매' '삼성메디슨' '삼성에스디아이'
 '삼성웰스토리' '삼성에스디에스' '삼성엔지니어링' '에스원' '호텔신라' '삼성화재해상보험' '삼성생명보험' '삼성자산운용'
 '삼성증권' '삼성카드' '삼성전자' '서울레이크사이드' '삼우종합건축사사무소' '씨브이네트' '삼성바이오로직스'
 '삼성코닝어드밴스드글라스' '에스유머티리얼스' '스테코' '삼성전자서비스씨에스' '삼성전자로지텍' '수원삼성축' '삼성전기'
 '삼성중공업' '제일패션리테일' '시큐아이' '멀티캠퍼스' '에스티엠' '에스코어' '미라콤아이앤씨' '제일기획' '신라스테이'
 '에이치디씨신라면세점' '에스비티엠' '삼성경제연' '삼성라이온즈' '삼성화재애니카손해사정' '삼성화재서비스손해사정' '삼성선물'


['엘지' '엘지화학' '씨텍' '행복누리' '팜한농' '엘지하우시스' '엘지토스템비엠' '그린누리' '엘지생활건강' '코카콜라음료'
 '한국음료' '해태에이치티비' '더페이스샵' '크린소울' '씨앤피코스메틱스' '캐이엔아이' '에프엠지' '엘지파루크' '밝은누리'
 '태극제약' '루치펠로코리아' '엘지엠엠에이' '엘지전자' '엘지디스플레이' '나눔누리' '엘지이노텍' '이노위드' '하이프라자'
 '하이엠솔루텍' '하이텔레서비스' '에이스냉동공조' '하누리' '로보스타' '로보메디' '실리콘웍스' '엘지유플러스' '씨에스리더'
 '아인텔레서비스' '미디어로그' '데이콤크로싱' '씨에스원파트너' '위드유' '엘지씨엔에스' '비즈테크파트너스' '행복마루'
 '에스앤아이코퍼레이션' '곤지암예원' '미래엠' '드림누리' '에스앤아이씨엠' '엘지경영개발원' '엘지스포츠' '지투알'
 '에이치에스애드' '엘베스트' '엘지상사' '판토스' '판토스부산신항물류센터' '헬리스타항공' '한울타리']
['엘지화학' '엘지하우시스' '엘지토스템비엠' '엘지생활건강' '더페이스샵' '엘지파루크' '엘지엠엠에이' '엘지전자'
 '엘지디스플레이' '엘지이노텍' '하이프라자' '엘지히타치워터솔루션' '엘지유플러스' '엘지씨엔에스' '에스앤아이코퍼레이션'
 '엘지경영개발원' '엘지스포츠' '에이치에스애드' '엘지상사' '판토스' 'LGHoldingsJapanCo.,Ltd' '엘지'
 '씨텍' '행복누리' '팜한농' '우지막코리아' '그린누리' '코카콜라음료' '한국음료' '해태에이치티비' '크린소울'
 '씨앤피코스메틱스' '캐이엔아이' '에프엠지' '미젠스토리' '밝은누리' '태극제약' '제이에스제약'
 '울릉추산용천수먹는샘물개발추진' '루치펠로코리아' '울릉샘물' '나눔누리' '이노위드' '하이엠솔루텍' '하이텔레서비스'
 '에이스냉동공조' '하이엔텍' '하누리' '엘지퓨얼셀시스템즈코리아' '로보스타' '로보메디' '실리콘웍스' '씨에스리더'
 '아인텔레서비스' '미디어로그' '데이콤크로

['지에스' '지에스에너지' '지에스파워' '보령엘엔지터미널' '지에스파크이십사' '지에스칼텍스' '지에스엠비즈' '이노폴리텍'
 '상지해운' '지에스바이오' '지에스에코메탈' '지에스리테일' '후레쉬서브' '지에스넷비전' '씨브이에스넷' '지에스네트웍스'
 '에피스' '지에스홈쇼핑' '지에스텔레서비스' '텐바이텐' '지에스글로벌' '피엘에스' '지에스엔텍' '지에스스포츠' '지에스건설'
 '파르나스호텔' '자이에스앤디' '자이에스텍' '지씨에스' '비에스엠' '자이에너지운영' '피앤에쓰' '지앤엠에스테이트'
 '지에스네오텍' '삼양통상' '삼양인터내셔날' '보헌개발' '켐텍인터내셔날' '옥산유통' '경원건설' '승산' '삼정건업'
 '지에스이앤알' '지에스풍력발전']
['지에스에너지' '지에스파워' '지에스파크이십사' '지에스칼텍스' '지에스엠비즈' '지에스바이오' '지에스에코메탈' '지에스리테일'
 '지에스넷비전' '지에스네트웍스' '지에스홈쇼핑' '지에스텔레서비스' '지에스이피에스' '지에스글로벌' '지에스엔텍'
 '지에스스포츠' '지에스건설' '지에스네오텍' '위너셋' '지에스이앤알' '지에스동해전력' '지에스풍력발전' '지에스포천그린에너지'
 '인천종합에너지' '자이에너지운영' '피앤에쓰' '보령엘엔지터미널' '이노폴리텍' '상지해운' '삼양인터내셔날' '켐텍인터내셔날'
 'GSCaltexChinaCo.,Ltd' 'GSCaltexTrading(Suzhou)Co.,Ltd' 'GSCCzechs.r.o'
 'GSC(Suzhou)PlasticsCo.,Ltd' 'GSCaltexIndiaPte.,Ltd'
 'GSCaltex(Langfang)PlasticsCo.,Ltd' 'GSCaltexSingaporePte.,Ltd'
 'GSLubricants(Tianjin)Co.,Ltd' 'GSCaltexMexicoS.deR.L.deC.V'
 'GSGLOBALSINGAPOREPTE.LTD' 'GSGLOBALUSA,INC' 'GSGlobalJapanCo.,LTD'
 'SYENE

['대한항공' '한진칼' '한진' '한국공항' '진에어' '정석기업' '싸이버스카이' '한진정보통신' '부산글로벌물류센터'
 '포항항7부두운영' '칼호텔네트워크' '항공종합서비스' '한진관광' '왕산레저개발' '서울복합물류프로젝트금융투자'
 '서울복합물류자산관리' '아이에이티' '에어코리아' '토파스여행정보' '한국글로발로지스틱스시스템' '한진울산신항운영'
 '한진인천컨테이너터미널' '평택컨테이너터미날' '한진부산컨테이너터미널' '태일통상' '태일캐터링' '청원냉장' '세계혼재항공화물'
 '더블유에이씨항공서비스']
['한진칼' '한진' '한국공항' '진에어' '싸이버스카이' '한진정보통신' '칼호텔네트워크' '항공종합서비스' '제동레저'
 '한진관광' '왕산레저개발' '아이에이티' '에어코리아' '토파스여행정보' '한국글로발로지스틱스시스템' '세계혼재항공화물'
 '더블유에이씨항공서비스' 'HanjinInternationalCorp' 'HanjinInternationalJapan'
 'TotalAviationServiceLLC' 'HanjinIntermodalAmericaInc'
 "WACINT'LLOGISTICSVIETNAMCONPANYLIMITED" '대한항공' '정석기업' '부산글로벌물류센터'
 '서울복합물류자산관리' '한국티비티' '한진울산신항운영' '한진인천컨테이너터미널' '평택컨테이너터미날' '한진부산컨테이너터미널'
 '더원에너지' 'HanjinGlobalLogistics(HongKong)Ltd'
 'HanjinGlobalLogistics(Dalian)Ltd'
 'QINGDAOHANJINLUHAIINTERNATIONALOGISTICSCO.,LTD'
 'HANJINGLOBALLOGISTICS(SHANGHAI)LIMITED'
 'HANJINGLOBALLOGISTICS(SHENZHEN)LIMITED'
 'HANJINGLOBALLOGISTICSEUROPEs.r.o' 'HANJINGLOBALLOGISTICSMYANMARCO.,LTD'
 'HANJINGLOBAL

['미래에셋캐피탈' '미래에셋대우' '미래에셋생명보험' '미래에셋자산운용' '미래에셋벤처투자' '미래에셋펀드서비스'
 '미래에셋금융서비스' '멀티에셋자산운용' '에너지인프라자산운용' '미래에셋파트너스제9호사모투자' '오딘제9차' '미래에셋컨설팅'
 '브랜드무브' '와이케이디벨롭먼트' '육공공']
['미래에셋대우' '미래에셋생명보험' '미래에셋글로벌유니콘사모투자' '미래에셋네이버아시아그로쓰사모투자'
 'MiraeAssetSecurities(Vietnam)LLC' 'MiraeAssetFinanceCompany(Vietnam)'
 '미래에셋캐피탈' '미래에셋자산운용' '미래에셋벤처투자' '멀티에셋자산운용' '미래에셋펀드서비스' '미래에셋금융서비스'
 '에너지인프라자산운용' '미래에셋파트너스9호' '미래에셋대우코리아제일호사모투자' '미래에셋대우글로벌케이그로쓰사모투자'
 '미래에셋대우오퍼튜너티사모투자' '미래에셋대우헌터스사모투자' '미래에셋대우글로벌제일호사모투자' '미래에셋대우코리아제이호사모투자'
 '미래에셋컨설팅' '브랜드무브' '수원학교사랑' '와이케이디벨롭먼트' 'MiraeAssetSecurities(HK)Limited'
 'MiraeAssetSecurities(UK)Ltd' '미래에셋모바일' '서울공항리무진'
 'MiraeAssetGlobalInvestments(HongKong)Ltd'
 'MiraeAssetGlobalInvestments(USA)INC'
 'MiraeAssetGlobalETFsHodlignsLimited' '오딘제9차']
      매출회사                              매입회사      매출액  국내계열사계(매출액)  \
0  미래에셋캐피탈                            미래에셋대우  31736.0      43772.0   
1  미래에셋캐피탈                          미래에셋생명보험   4692.0      43772.0   
2  미래에셋캐피탈                    

...

['케이씨씨' '케이씨씨건설' '코리아오토글라스' '금강레저' '세우실업' '동주상사' '동주' '대호포장' '티앤케이정보'
 '주령금속' '모멘티브퍼포먼스머티리얼스코리아']
['케이씨씨건설' '코리아오토글라스' '세우실업' 'KCS' 'KCK' 'KCB' 'KCG' 'KCQ' 'KCT' 'KCI'
 'KCID' 'KCV' 'KCC(Hanoi)' 'KCCPaint' 'Basildon' '케이씨씨' '동주'
 'MomentivePerformanceMaterialsAsiaPacificPteLtd'
 'MomentivePerformanceMaterialsGmbH'
 'MomentivePerformanceMaterials(India)PrivateLimited'
 'MomentivePerformanceMaterials(NanTong)CO.,LTD'
 'MomentivePerformanceMaterials(Thailand)Ltd'
 'MomentivePerformanceMaterialsHongKongLtd'
 'MomentivePerformanceMaterialsJapanLLC' 'MPMSiliconesLLC'
 'MomentivePerformanceMaterialsUSALLC'
 'MomentivePerformanceMaterialsQuartzInc']
   매출회사      매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)  해외계열사계(매출액)      국내매출액  \
0  케이씨씨    케이씨씨건설   19326.0     134286.0      99421.0          NaN  1816508.0   
1  케이씨씨  코리아오토글라스  114949.0     134286.0      99421.0          NaN  1816508.0   
2  케이씨씨      세우실업      11.0     134286.0      99421.0          NaN  1816508.0   
3  케이씨씨       KCS      93.

['남선알미늄' '남선홀딩스' '바로코사' '바로코사유통사하' '벡셀' '삼라' '삼라마이다스' '우방' '울산방송' '이엔에이치'
 '케이엘홀딩스' '티케이케미칼' '한통엔지니어링' '대한상선' '대한해운' '동아건설산업' '삼환기업' '에스엠레저산업'
 '에스엠상선' '에스엠상선광양터미널' '에스엠스틸' '에스엠신용정보' '에스엠인더스트리' '에스엠중공업' '에스엠하이플러스'
 '이코사주류' '케이엘씨에스엠' '코리코엔터프라이스' '플러스매니지먼트' '한국선박금융' '한덕철광산업' '에스엠에이엠씨투자대부']
['삼라' '우방' '티케이케미칼' '동아건설산업' '에스엠상선' '에스엠인더스트리' '바로코사유통사하' '벡셀' '에스엠하이플러스'
 '에스엠생명과학' '바로코사' '남선알미늄' '태길종합건설' '경남기업' '대한상선' '대한해운' '삼환기업'
 '에스엠상선경인터미널' '에스엠스틸' '에스엠신용정보' '에스엠중공업' '이코사주류' '케이엘씨에스엠' '코리코엔터프라이스'
 '한덕철광산업' '신광' '서림하이팩' '에스엠케미칼' '삼라희망재단' '에스엠에이엠씨투자대부' '케이엘홀딩스' '에스엠티케미칼'
 '삼라홀딩스' '케이엘홀딩스이호' '우방산업' '신촌역사' '이엔에이치' '삼라농원' '코리아엘앤지트레이딩' '플러스매니지먼트'
 '한일개발' '라도' '신광하이메탈' '판교에스디쓰리' '한국선박금융' '기원토건' '서남바이오에너지' '에스엠레저산업'
 '동강시스타' '대림종합건설더블유']
    매출회사    매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액     해외매출액
0  남선알미늄      삼라   561.0      15021.0          NaN  216430.0  108352.0
1  남선알미늄      우방  6225.0      15021.0          NaN  216430.0  108352.0
2  남선알미늄  티케이케미칼    66.0      15021.0          NaN 

['호반건설' '호반자산개발' '호반산업' '티에스자산개발' '호반티비엠' '호반프라퍼티' '호반호텔앤리조트' '그린산업'
 '퍼시픽마리나' '플랜에이치벤처스' '호반서서울' '광주방송' '케이비씨플러스' '호반써밋' '아브뉴프랑' '호반스카이밸리'
 '코너스톤투자파트너스']
['호반자산개발' '대전용산개발' '스카이리빙' '스카이하우징' '호반주택' '호반산업' '배곧랜드마크피에프브이' '호반호텔앤리조트'
 '광주방송' '호반건설' '에이치비탕정' '티에스주택' '티에스개발' '티에스건설' '티에스자산개발' '티에스리빙' '호반티비엠'
 '호반프라퍼티' '플랜에이치벤처스' '호반써밋' '아브뉴프랑' '호반스카이밸리' '코너스톤투자파트너스' '호반서서울'
 '케이비씨플러스' '대아청과' '리젠시빌주택' '리젠시빌건설']
   매출회사    매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)      국내매출액  해외매출액
0  호반건설  호반자산개발  28291.0     284472.0          NaN  1977191.0    NaN
1  호반건설  대전용산개발    132.0     284472.0          NaN  1977191.0    NaN
2  호반건설   스카이리빙  16258.0     284472.0          NaN  1977191.0    NaN
3  호반건설  스카이하우징  16454.0     284472.0          NaN  1977191.0    NaN
4  호반건설    호반주택  10466.0     284472.0          NaN  1977191.0    NaN
           매출회사        매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액  해외매출액
106     호반스카이밸리        광주방송   16.0       1457.0          NaN  21097.0    NaN
107     호반스카이밸리        호

['한라홀딩스' '위코' '제이제이한라' '한라' '한라세라지오' '한라지엘에스' '한라오엠에스' '만도' '만도차이나홀딩스'
 '만도헬라일렉트로닉스' '만도브로제']
['위코' '제이제이한라' '한라' '한라세라지오' '한라지엘에스' '한라오엠에스' '한라개발' '목포신항만운영' '만도'
 '만도헬라일렉트로닉스' '만도브로제' '한라홀딩스' '배곧신도시지역특성화타운' '평택동부도로' '만도차이나홀딩스']
    매출회사    매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액    해외매출액
0  한라홀딩스      위코     15.0     148526.0     106564.0  564680.0  55923.0
1  한라홀딩스  제이제이한라     52.0     148526.0     106564.0  564680.0  55923.0
2  한라홀딩스      한라  20895.0     148526.0     106564.0  564680.0  55923.0
3  한라홀딩스  한라세라지오     25.0     148526.0     106564.0  564680.0  55923.0
4  한라홀딩스  한라지엘에스    441.0     148526.0     106564.0  564680.0  55923.0
          매출회사        매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액  \
35  만도헬라일렉트로닉스          만도  422963.0     422986.0      54147.0  454975.0   
36  만도헬라일렉트로닉스       만도브로제      23.0     422986.0      54147.0  454975.0   
37       만도브로제       한라홀딩스   14679.0      83176.0       6679.0   69627.0   
38       만도브로제          만도   47910.0     

['금호석유화학' '금호피앤비화학' '금호미쓰이화학' '금호폴리켐' '금호티앤엘' '금호개발상사' '여수페트로']
['금호피앤비화학' '금호미쓰이화학' '금호폴리켐' '금호개발상사' '금호티앤엘' '코리아에너지발전소' '상해금호일려소료유한공사'
 '광동금호일려고분자재료유한공사' '남경금포금호화공유한공사' '금호석유화학' '강소금호양농화공유한공사']
     매출회사     매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)      국내매출액      해외매출액
0  금호석유화학  금호피앤비화학  153335.0     189833.0      20242.0  1220149.0  2478974.0
1  금호석유화학  금호미쓰이화학    8681.0     189833.0      20242.0  1220149.0  2478974.0
2  금호석유화학    금호폴리켐   23178.0     189833.0      20242.0  1220149.0  2478974.0
3  금호석유화학   금호개발상사     185.0     189833.0      20242.0  1220149.0  2478974.0
4  금호석유화학    금호티앤엘    4435.0     189833.0      20242.0  1220149.0  2478974.0
      매출회사     매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액   해외매출액
21  금호개발상사  금호미쓰이화학     3.0      18413.0          NaN  48960.0  1283.0
22  금호개발상사    금호폴리켐     4.0      18413.0          NaN  48960.0  1283.0
23  금호개발상사    금호티앤엘  4398.0      18413.0          NaN  48960.0  1283.0
24   여수페트로   금호석유화학    64.0        229.0          NaN   12

In [4]:
myApi.getTransTableAll(2021)

===============삼성===============
['삼성전자' '삼성물산' '삼우종합건축사사무소' '씨브이네트' '삼성바이오로직스' '삼성디스플레이' '삼성코닝어드밴스드글라스'
 '에스유머티리얼스' '스테코' '세메스' '삼성전자서비스' '삼성전자서비스씨에스' '삼성전자판매' '삼성전자로지텍' '수원삼성축'
 '삼성메디슨' '삼성에스디아이' '삼성전기' '삼성중공업' '에스디플렉스' '제일패션리테일' '삼성웰스토리' '시큐아이'
 '멀티캠퍼스' '에스티엠' '삼성에스디에스' '에스코어' '오픈핸즈' '미라콤아이앤씨' '삼성엔지니어링' '에스원'
 '휴먼티에스에스' '에스원씨알엠' '제일기획' '호텔신라' '신라스테이' '에스비티엠' '삼성경제연' '삼성라이온즈'
 '삼성화재해상보험' '삼성화재애니카손해사정' '삼성화재서비스손해사정' '삼성화재금융서비스보험대리점' '삼성생명보험' '삼성선물'
 '삼성자산운용' '삼성액티브자산운용' '삼성생명서비스손해사정' '삼성에스알에이자산운용' '삼성생명금융서비스보험대리점' '삼성증권'
 '삼성카드' '삼성카드고객서비스']
['삼성물산' '삼성바이오에피스' '삼성디스플레이' '삼성전자서비스' '삼성전자판매' '삼성메디슨' '삼성에스디아이' '삼성전기'
 '삼성웰스토리' '삼성에스디에스' '삼성엔지니어링' '에스원' '호텔신라' '삼성화재해상보험' '삼성증권' '삼성카드' '삼성전자'
 '서울레이크사이드' '삼우종합건축사사무소' '씨브이네트' '삼성바이오로직스' '삼성코닝어드밴스드글라스' '에스유머티리얼스'
 '스테코' '삼성전자로지텍' '수원삼성축' '삼성중공업' '제일패션리테일' '멀티캠퍼스' '에스티엠' '에스코어' '미라콤아이앤씨'
 '제일기획' '에이치디씨신라면세점' '에스비티엠' '삼성경제연' '삼성라이온즈' '삼성화재애니카손해사정' '삼성화재서비스손해사정'
 '삼성생명보험' '삼성선물' '삼성자산운용' '삼성액티브자산운용' '삼성카드고객서비스' '삼성벤처투자' '삼성전자서비스씨에스'
 '신라스테이' '삼성

['엘지' '엘지화학' '씨텍' '행복누리' '팜한농' '엘지하우시스' '그린누리' '엘지생활건강' '코카콜라음료' '한국음료'
 '해태에이치티비' '에프엠지' '엘지파루크' '미젠스토리' '밝은누리' '태극제약' '루치펠로코리아' '엘지엠엠에이' '엘지전자'
 '엘지디스플레이' '나눔누리' '엘지이노텍' '이노위드' '하이프라자' '하이엠솔루텍' '하이텔레서비스' '에이스냉동공조'
 '하누리' '로보스타' '로보메디' '실리콘웍스' '엘지유플러스' '씨에스리더' '아인텔레서비스' '미디어로그' '데이콤크로싱'
 '씨에스원파트너' '위드유' '엘지헬로비전' '유플러스홈서비스' '엘지씨엔에스' '비즈테크파트너스' '행복마루'
 '에스앤아이코퍼레이션' '곤지암예원' '드림누리' '에스앤아이씨엠' '엘지경영개발원' '엘지스포츠' '지투알' '에이치에스애드'
 '엘베스트' '엘지상사' '판토스' '판토스부산신항물류센터' '헬리스타항공' '한울타리']
['엘지화학' '엘지에너지솔루션' '엘지하우시스' '엘지토스템비엠' '엘지생활건강' '엘지파루크' '더페이스샵' '엘지엠엠에이'
 '엘지전자' '엘지디스플레이' '엘지이노텍' '하이프라자' '엘지유플러스' '엘지헬로비전' '엘지씨엔에스' '에스앤아이코퍼레이션'
 '엘지경영개발원' '엘지스포츠' '에이치에스애드' '엘지상사' '판토스' 'LGHoldingsJapanCo.,Ltd' '엘지'
 '씨텍' '행복누리' '팜한농' '코카콜라음료' '해태에이치티비' '로보스타' '그린누리' '곤지암예원' '한국음료' '크린소울'
 '에프엠지' '미젠스토리' '밝은누리' '태극제약' '루치펠로코리아' '캐이앤아이' '씨앤피코스메틱스' '유플러스홈서비스'
 '이노위드' '씨에스리더' '비즈테크파트너스' '미래엠' '하이엠솔루텍' '하이텔레서비스' '하누리' '실리콘웍스'
 '아인텔레서비스' '미디어로그' '씨에스원파트너' '위드유' '지투알' '엘베스트' '판토스부산신항물류센터' '에이스냉동공조'
 '우지막코리아' '울릉샘물' '로아코리

['지에스' '지에스에너지' '지에스파워' '보령엘엔지터미널' '지에스파크이십사' '지에스칼텍스' '지에스엠비즈' '이노폴리텍'
 '상지해운' '지에스바이오' '지에스에코메탈' '한울사랑' '지에스리테일' '후레쉬서브' '지에스넷비전' '지에스네트웍스'
 '후레쉬미트' '지에스홈쇼핑' '지에스텔레서비스' '지에스글로벌' '피엘에스' '지에스엔텍' '지에스스포츠' '지에스건설'
 '파르나스호텔' '자이에스앤디' '자이에스텍' '지씨에스' '비에스엠' '자이에너지운영' '피앤에쓰' '지앤엠에스테이트'
 '지에스네오텍' '삼양통상' '삼양인터내셔날' '보헌개발' '켐텍인터내셔날' '경원건설' '승산' '삼정건업' '지에스이앤알'
 '지에스풍력발전' '동해그린에너지']
['지에스에너지' '지에스파워' '지에스파크이십사' '지에스칼텍스' '지에스엠비즈' '지에스바이오' '지에스에코메탈' '지에스리테일'
 '지에스넷비전' '지에스네트웍스' '지에스홈쇼핑' '지에스텔레서비스' '지에스이피에스' '지에스글로벌' '지에스엔텍'
 '지에스스포츠' '지에스건설' '지에스네오텍' '위너셋' '지에스이앤알' '지에스동해전력' '지에스풍력발전' '지에스포천그린에너지'
 '인천종합에너지' 'KoreaGSE&PPte.Ltd' 'GSEnergyTradingSingaporePte.Ltd' '삼양인터내셔날'
 '파르나스호텔' '자이에너지운영' '피앤에쓰' '지에스' '이노폴리텍' '상지해운' '켐텍인터내셔날'
 'GSCaltexChinaCo.,Ltd' 'GSCCzechs.r.o' 'GSC(Suzhou)PlasticsCo.,Ltd'
 'GSCaltexIndiaPte.,Ltd' 'GSCaltex(Langfang)PlasticsCo.,Ltd'
 'GSCaltexSingaporePte.,Ltd' 'GSLubricants(Tianjin)Co.,Ltd'
 'GSCaltexMexicoS.deR.L.deC.V' 'GSGLOBALSINGAPOREPTE.LTD'
 'GSAromaticsPte.Ltd' 'Qing

['대한항공' '한진칼' '한진' '한국공항' '정석기업' '한진관광' '토파스여행정보' '한진정보통신' '진에어' '싸이버스카이'
 '에어코리아' '칼호텔네트워크' '한국글로발로지스틱스시스템' '한진인천컨테이너터미널' '항공종합서비스' '포항항7부두운영'
 '부산글로벌물류센터' '평택컨테이너터미날' '한진부산컨테이너터미널' '아이에이티' '한진울산신항운영' '서울복합물류자산관리'
 '서울복합물류프로젝트금융투자' '왕산레저개발' '태일통상' '태일캐터링' '청원냉장' '세계혼재항공화물' '더블유에이씨항공서비스']
['한진칼' '한진' '진에어' '정석기업' '칼호텔네트워크' '토파스여행정보' '한진관광' '한국공항' '한진정보통신'
 '항공종합서비스' '싸이버스카이' '한국글로발로지스틱스시스템' '아이에이티' '왕산레저개발' '에어코리아' '세계혼재항공화물'
 '더블유에이씨항공서비스' '제동레저' 'HanjinInternationalCorp' "HanjinInt'lJapanCo.,Ltd"
 'TotalAviationServiceLLC' 'HanjinIntermodalAmericaInc' '대한항공'
 '한진부산컨테이너터미널' '한진인천컨테이너터미널' '평택컨테이너터미날' '부산글로벌물류센터' '한진울산신항운영'
 '서울복합물류자산관리' '한국티비티' 'QINGDAOHANJINLUHAIINTERNATIONALLOGISTICSCO.,LTD'
 'HanjinGlobalLogistics(HongKong)Ltd'
 'HANJINGLOBALLOGISTICS(SHANGHAI)LIMITED'
 'HanjinGlobalLogistics(Dalian)Ltd'
 'HANJINGLOBALLOGISTICS(SHENZHEN)LIMITED'
 'HANJINGLOBALLOGISTICSEUROPEs.r.o' 'HANJINGLOBALLOGISTICSMYANMARCO.,LTD'
 'HANJINGLOBALLOGISTICS(VIETNAM)LIMITED' '포항항7부두운영' '태일캐터링'
 'Glo

['디엘' '대림' '디엘건설' '디엘모터스' '글래드호텔앤리조트' '에이플러스디' '제주항공우주호텔' '디엘에너지' '하이웨이솔라'
 '포승그린파워' '청진이삼프로젝트' '청진이삼자산관리' '대림에이엠씨' '대림피앤피'
 '대림제7호마산회원기업형임대위탁관리부동산투자회사']
['대림' '디엘건설' '대림씨엔에스' '글래드호텔앤리조트' '송도파워' '에코술이홀' '험프리에스엘큐원' '제주항공우주호텔'
 '덕송내각고속화도로' '디엘에너지' '청진이삼자산관리' '대림에이엠씨' '대림오토바이' '대림피앤피'
 '대림제7호마산회원기업형임대위탁관리부동산투자회사' '디엘에프엔씨' '효제피에프브이' '디엘' '디엘모터스' '포천파워'
 '에이플러스디' '밀머란에스피씨' '포승그린파워' '코크레인에스피씨' '청진이삼프로젝트'
 '대림제6호부산우암동기업형임대위탁관리부동산투자회사']
  매출회사       매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)      국내매출액     해외매출액
0   디엘         대림   4843.0     677705.0      47040.0  6181343.0  936791.0
1   디엘       디엘건설  83395.0     677705.0      47040.0  6181343.0  936791.0
2   디엘     대림씨엔에스      3.0     677705.0      47040.0  6181343.0  936791.0
3   디엘  글래드호텔앤리조트   1809.0     677705.0      47040.0  6181343.0  936791.0
4   디엘       송도파워      2.0     677705.0      47040.0  6181343.0  936791.0
                         매출회사    매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)  \
55                      대림피앤피      디엘  2106.0   

...

['티케이케미칼' '남선알미늄' '남선홀딩스' '라도' '바로코사' '벡셀' '삼라' '삼라농원' '삼라마이다스' '우방'
 '울산방송' '이엔에이치' '케이엘홀딩스' '한통엔지니어링' '대한상선' '대한해운' '동아건설산업' '삼환기업'
 '에스엠글로벌유통' '에스엠레저산업' '에스엠상선' '에스엠스틸' '에스엠신용정보' '에스엠인더스트리' '에스엠중공업'
 '에스엠하이플러스' '에스엠화진' '에스엠화진인터내셔널' '우방산업' '이코사주류' '케이엘씨에스엠' '코리코엔터프라이스'
 '탑스텐동강시스타' '플러스매니지먼트' '한덕철광산업' '에스엠에이엠씨투자대부']
['삼라' '우방' '이엔에이치' '경남기업' '대한해운' '동아건설산업' '에스엠인더스트리' '에스엠중공업' '에스엠하이플러스'
 '티케이케미칼' '삼환기업' '에스엠상선' '벡셀' '에스엠글로벌유통' '코리코엔터프라이스' '남선알미늄' '바로코사' '우방산업'
 '라도' '케이엘홀딩스' '한통엔지니어링' '대한상선' '대한해운엘엔지' '신화디앤디' '에스엠스틸' '에스엠신용정보' '삼라농원'
 'SMVNCO.,LTD' '태길종합건설' '에스엠상선경인터미널' '케이엘씨에스엠' '한국선박금융' '한덕철광산업'
 '에스엠에이엠씨투자대부' '삼라마이다스' '에스엠화진' '이코사주류' '에스엠화진인터내셔널' '탑스텐동강시스타' '판교에스디쓰리'
 '서남바이오에너지' '에스엠레저산업' '화진고신장식재료' '대림종합건설더블유']
     매출회사   매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액    해외매출액
0  티케이케미칼     삼라    19.0       3976.0          NaN  331911.0  79116.0
1  티케이케미칼     우방     1.0       3976.0          NaN  331911.0  79116.0
2  티케이케미칼  이엔에이치  3683.0       3976.0          NaN  331911.0  7

['디엠씨미디어' '리앤에스스포츠' '무노스' '문고리닷컴' '블루원' '블루원레저' '에스비에스' '에스비에스네오파트너스'
 '에스비에스디지털뉴스랩' '에스비에스미디어넷' '에스비에스아이앤엠' '에스비에스에이앤티' '에스비에스엠앤씨' '에스비에스콘텐츠허브'
 '에스비에스플러스' '엠시에타' '이너시티개발' '인제스피디움' '태영건설' '태영그레인터미널' '티에스케이그린에너지'
 '티에스케이엔지니어링' '티에스케이엠엔에스' '티에스케이워터' '티에스케이이앤피' '티에스케이코퍼레이션' '티에스케이프리텍'
 '평택싸이로' '포맷티스트' '삼계개발' '스마트미디어렙' '스튜디오에스' '신경주지역개발' '에스비에스미디어홀딩스'
 '에스비에스바이아컴' '에코시스템']
['에스비에스' '스마트미디어렙' '에스비에스미디어넷' '에스비에스엠앤씨' 'SBS디지털뉴스랩' '문고리닷컴' '블루원'
 '에스비에스아이앤엠' 'SBS플러스' '에스비에스콘텐츠허브' 'SBS미디어홀딩스' '태영건설' '태영인더스트리' '블루원레저'
 '엠시에타' '에스비에스바이아컴' '스튜디오에스' '디엠씨미디어' 'SBS에이엔티' 'SBSinternational' 'KCP'
 '블렌딩' '리앤에스스포츠' '무노스' '콘텐츠웨이브' '엠시에타개발' 'SBS아프리카TV'
 'KoreaContentPlatfrom,LCC' '에코시티' '티에스케이워터' '대AMC' '에코시티개발' '포천바이오에너지'
 '경산에코에너지' '신경주역세권공영개발' '이너시티개발' '평택싸이로' '네오시티' '군포복합개발PFV' 'TSK이앤피'
 'TSK코퍼레이션' '울산이앤피' '휴비스워터' '여수엑스포환경' '여천이피에스' '부산바이오에너지' 'TSK엔지니어링'
 'TSK엔워터테크' 'TSK그린에너지' '태영그레인터미널' '에스비에스네오파트너스']
     매출회사       매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액  해외매출액
0  디엠씨미디어      에스비에스   896.0   

['한라홀딩스' '위코' '제이제이한라' '한라' '한라세라지오' '한라지엘에스' '한라오엠에스' '만도' '만도헬라일렉트로닉스'
 '만도브로제']
['위코' '제이제이한라' '한라' '한라세라지오' '한라지엘에스' '한라오엠에스' '목포신항만운영' '만도' '만도헬라일렉트로닉스'
 '만도브로제' '한라홀딩스' '배곧신도시지역특성화타운' '평택동부도로' '만도차이나홀딩스']
    매출회사    매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액     해외매출액
0  한라홀딩스      위코    13.0     103905.0     105208.0  440230.0  127369.0
1  한라홀딩스  제이제이한라    40.0     103905.0     105208.0  440230.0  127369.0
2  한라홀딩스      한라  9789.0     103905.0     105208.0  440230.0  127369.0
3  한라홀딩스  한라세라지오    17.0     103905.0     105208.0  440230.0  127369.0
4  한라홀딩스  한라지엘에스   446.0     103905.0     105208.0  440230.0  127369.0
          매출회사        매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액  \
39  만도헬라일렉트로닉스          만도  436042.0     436281.0      58337.0  466144.0   
40  만도헬라일렉트로닉스       만도브로제      31.0     436281.0      58337.0  466144.0   
41       만도브로제       한라홀딩스    6677.0      69819.0       8457.0   81326.0   
42       만도브로제          만도   36745.0      69819.0       845

['장금상선' '와이제이씨' '한성라인' '경평물류' '광양항서부컨테이너터미널' '국양로지텍' '국양해운' '부산항터미널'
 '시노코쉽매니지먼트' '에스엠상선광양터미널' '장금마리타임' '조강해운' '퍼시픽엔지니어링' '흥아라인']
['와이제이씨' '한성라인' '경평물류' '국양로지텍' '국양해운' '슈퍼랙' '시노코쉽매니지먼트' '시노코탱커'
 '시노코페트로케미컬' '장금마리타임' '퍼시픽엔지니어링' '흥아라인'
 'TIANJINYUJIACHANGINTERNATIONALFORWARDING' '장금상선' '에스엠상선광양터미널'
 '광양항서부컨테이너터미널' '부산항터미널']
   매출회사   매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액     해외매출액
0  장금상선  와이제이씨   529.0     120650.0       3140.0  690953.0  518181.0
1  장금상선   한성라인  9807.0     120650.0       3140.0  690953.0  518181.0
2  장금상선   경평물류    23.0     120650.0       3140.0  690953.0  518181.0
3  장금상선  국양로지텍   566.0     120650.0       3140.0  690953.0  518181.0
4  장금상선   국양해운   754.0     120650.0       3140.0  690953.0  518181.0
    매출회사    매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액     해외매출액
69  흥아라인   와이제이씨    41.0      74404.0          NaN  248521.0  386468.0
70  흥아라인    한성라인  2050.0      74404.0          NaN  248521.0  386468.0
71  흥아라인    경평물류    39.0      74404.0          Na

['대방건설' '대방산업개발' '대덕하우징씨스템' '지유인터내셔날' '디비일산' '디케이일산']
['대방하우징' '대방주택' '노블랜드' '디비건설' '디비산업개발' '대방이노베이션' '대방이엔씨' '대방개발기업' '디비개발'
 '엔비건설' '디비종합건설' '디비주택' '대방일산디엠시티' '대방덕은' '디엠개발' '대방산업개발' '대방산업개발동탄'
 '대방건설' '엘리움' '엘리움개발' '엘리움건설' '엘리움주텍' '엘리움주택개발']
   매출회사    매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)      국내매출액  해외매출액
0  대방건설   대방하우징  153182.0     970711.0          NaN  1557457.0    NaN
1  대방건설    대방주택  103392.0     970711.0          NaN  1557457.0    NaN
2  대방건설    노블랜드   86208.0     970711.0          NaN  1557457.0    NaN
3  대방건설    디비건설  261352.0     970711.0          NaN  1557457.0    NaN
4  대방건설  디비산업개발   78658.0     970711.0          NaN  1557457.0    NaN
        매출회사      매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액  해외매출액
33  대덕하우징씨스템    대방산업개발    332.0       3063.0          NaN   8119.0    NaN
34   지유인터내셔날      대방건설    144.0        196.0          NaN    391.0    NaN
35   지유인터내셔날    대방산업개발     52.0        196.0          NaN    391.0    NaN
36      디비일산  대방일산디엠시티  10107.0      10

In [5]:
myApi.mergeTransId(2019)

=============== 삼성 ===============
[]
['에스프린팅솔루션']
   매출회사      매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액        해외매출액  \
0  삼성전자      삼성물산   78676      2269950  149436938.0  16821275  153560595.0   
1  삼성전자  삼성바이오에피스       1      2269950  149436938.0  16821275  153560595.0   
2  삼성전자   삼성디스플레이  164404      2269950  149436938.0  16821275  153560595.0   
3  삼성전자       세메스       5      2269950  149436938.0  16821275  153560595.0   
4  삼성전자   삼성전자서비스  456109      2269950  149436938.0  16821275  153560595.0   

         매출액총계  year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융       매출액Val  \
0  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   
1  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   
2  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   
3  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   
4  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   


=============== 엘지 ===============
=============== 롯데 ===============
[]
['롯데디에프글로벌', '롯데닷컴', '롯데백화점마산', '롯데시네마', '롯데로지스틱스', '롯데아이티테크']
   매출회사      매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액  해외매출액   매출액총계  \
0  롯데지주      호텔롯데  12284       167669          NaN  255644   7296  262940   
1  롯데지주   롯데면세점제주   1140       167669          NaN  255644   7296  262940   
2  롯데지주    부산롯데호텔   1073       167669          NaN  255644   7296  262940   
3  롯데지주      롯데물산    913       167669          NaN  255644   7296  262940   
4  롯데지주  롯데디에프글로벌      1       167669          NaN  255644   7296  262940   

   year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융    매출액Val        매입회사id  \
0   2019  2018       롯데  310204.0     상장       비금융  262940.0  6.300200e+05   
1   2019  2018       롯데  310204.0     상장       비금융  262940.0  1.801708e+09   
2   2019  2018       롯데  310204.0     상장       비금융  262940.0  6.306830e+05   
3   2019  2018       롯데  310204.0     상장       비금융  262940.0  6.208900e+05   
4   2019  2

=============== 신세계 ===============
=============== 한진 ===============
[]
['한국글로발로지스틱시스템', 'HANJININTERMODALAMERICAINC', 'HANJINGLOBALLOGISTICS(VIETNAM)Ltd', "HanjinInt'lCorp", "QINGDAOHANJINLUHAIINT'LLOGISTICSCO.,LTD", 'EURASIALOGISTICSSERVICELLC', 'HanjinGlobalLogistics(HongKong)Ltd', 'HANJINGLOBALLOGISTICS(SHANGHAI)LIMITED', 'HanjinGlobalLogistics(Dalian)Ltd', 'HANJINGLOBALLOGISTICS(SHENZHEN)LIMITED', 'HANJINGLOBALLOGISTICSGUANGZHOUCO.,LTD', 'HANJINGLOBALLOGISTICSEUROPEs.r.o', 'GLSAP', "WACINT'LLOGISTICS(HKLIMITED)", "WACINT'LLOGISTICS(QINGDAOCOMPANYLIMITED)"]
   매출회사  매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액       해외매출액  \
0  대한항공   한진칼   178       199900       1714.0  1183532  11463333.0   
1  대한항공    한진  3347       199900       1714.0  1183532  11463333.0   
2  대한항공  한국공항  1003       199900       1714.0  1183532  11463333.0   
3  대한항공  정석기업     1       199900       1714.0  1183532  11463333.0   
4  대한항공  한진관광   112       199900       1714.0  1183532  11463333.0   

        매

=============== 대림 ===============
=============== 미래에셋 ===============
['미래에셋파트너스제구호사모투자']
['MiraeAssetFinanceCompany(Vietnam)', 'PCA생명보험', '미래에셋파트너스5호', '미래에셋파트너스제구호사모투자', 'MiraeAssetSecurities(HK)Limited', '미래에셋파트너스칠호사모투자', 'MiraeAssetGlobalInvestments(HK)Limited', 'MiraeAssetGlobalInvestments(USA)INC', 'MiraeAssetGlobalInvestments(Taiwan)', 'MiraeAssetGlobalETFsHodlignsLimited']
      매출회사       매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액   해외매출액  \
0  미래에셋캐피탈     미래에셋대우  29421        37850       2231.0  174685  2231.0   
1  미래에셋캐피탈   미래에셋자산운용     12        37850       2231.0  174685  2231.0   
2  미래에셋캐피탈   미래에셋생명보험   4692        37850       2231.0  174685  2231.0   
3  미래에셋캐피탈   멀티에셋자산운용      5        37850       2231.0  174685  2231.0   
4  미래에셋캐피탈  미래에셋금융서비스      2        37850       2231.0  174685  2231.0   

      매출액총계  year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융    매출액Val  \
0  176916.0   2019  2018     미래에셋  965251.0    비상장        금융  176916.0   
1  176916.0   2019  2

=============== 하림 ===============
=============== 교보생명보험 ===============
[]
['교보생명자산운용-미국']
     매출회사          매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액  해외매출액  \
0  교보생명보험  교보라이프플래닛생명보험     8        16135          NaN  12505561    NaN   
1  교보생명보험      교보악사자산운용  1362        16135          NaN  12505561    NaN   
2  교보생명보험          교보증권   478        16135          NaN  12505561    NaN   
3  교보생명보험       생보부동산신탁    52        16135          NaN  12505561    NaN   
4  교보생명보험       생보부동산신탁    52        16135          NaN  12505561    NaN   

   매출액총계  year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융      매출액Val  \
0    NaN   2019  2018   교보생명보험  870200.0    비상장        금융  12505561.0   
1    NaN   2019  2018   교보생명보험  870200.0    비상장        금융  12505561.0   
2    NaN   2019  2018   교보생명보험  870200.0    비상장        금융  12505561.0   
3    NaN   2019  2018   교보생명보험  870200.0    비상장        금융  12505561.0   
4    NaN   2019  2018   교보생명보험  870200.0    비상장        금융  12505561.0   

       매입회사

=============== 케이씨씨 ===============
=============== SM ===============
[]
['에이디엠이십일', '대원건설산업', '성우종합건설', '온양관광호텔', '삼라희망재단', '삼라자원개발']
    매출회사    매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액     해외매출액     매출액총계  \
0  남선알미늄      우방  5864        16306          NaN  245704  102131.0  347835.0   
1  남선알미늄  티케이케미칼   547        16306          NaN  245704  102131.0  347835.0   
2  남선알미늄    경남모직   371        16306          NaN  245704  102131.0  347835.0   
3  남선알미늄  동아건설산업  5193        16306          NaN  245704  102131.0  347835.0   
4  남선알미늄   에스엠상선  3507        16306          NaN  245704  102131.0  347835.0   

   year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융    매출액Val      매입회사id  \
0   2019  2018       SM  380040.0     상장       비금융  347835.0  13071534.0   
1   2019  2018       SM  380040.0     상장       비금융  347835.0  12099515.0   
2   2019  2018       SM  380040.0     상장       비금융  347835.0    768600.0   
3   2019  2018       SM  380040.0     상장       비금융  347835.0    500135.0   
4 

['디비Inc']
['디비Inc', '대우전자', '대우전자서비스', '디비라이텍', '디비MetalUSAINC', '디비MetalEuropeGmbH']
    매출회사      매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액  해외매출액   매출액총계  \
0  디비Inc     디비Inc      0        46302            0  122170  84669  206839   
1  디비Inc      디비메탈   6651        46302            0  122170  84669  206839   
2  디비Inc      디비월드    158        46302            0  122170  84669  206839   
3  디비Inc     디비하이텍  10506        46302            0  122170  84669  206839   
4  디비Inc  디비에프아이에스   1138        46302            0  122170  84669  206839   

   year2  year grpname2  매출회사id 매출사상/비 매출사금융/비금융  매출액Val      매입회사id 매입사상/비  \
0   2019  2018       DB     NaN    NaN       NaN     NaN         NaN    NaN   
1   2019  2018       DB     NaN    NaN       NaN     NaN  12097776.0    비상장   
2   2019  2018       DB     NaN    NaN       NaN     NaN    910813.0    비상장   
3   2019  2018       DB     NaN    NaN       NaN     NaN    350354.0     상장   
4   2019  2018       DB     NaN    NaN       NaN  

=============== 아모레퍼시픽 ===============
=============== 삼천리 ===============
[]
['PT.SAMINDOUTAMAKALTIM', 'PT.TRASINDOMURNIPERKASA']
  매출회사    매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액  해외매출액  매출액총계  year2  \
0  삼천리  삼천리이엔지  11297      18838.0          NaN  2534707    NaN    NaN   2019   
1  삼천리  삼천리이에스    424      18838.0          NaN  2534707    NaN    NaN   2019   
2  삼천리  삼천리모터스      4      18838.0          NaN  2534707    NaN    NaN   2019   
3  삼천리     휴세스      5      18838.0          NaN  2534707    NaN    NaN   2019   
4  삼천리    에스파워   4929      18838.0          NaN  2534707    NaN    NaN   2019   

   year grpname2    매출회사id 매출사상/비 매출사금융/비금융     매출액Val      매입회사id 매입사상/비  \
0  2018      삼천리  350010.0     상장       비금융  2534707.0    195935.0    비상장   
1  2018      삼천리  350010.0     상장       비금융  2534707.0    189291.0    비상장   
2  2018      삼천리  350010.0     상장       비금융  2534707.0    557191.0    비상장   
3  2018      삼천리  350010.0     상장       비금융  2534707.0    483459.0    비상장 

=============== 다우키움 ===============


(       매출회사      매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)       국내매출액  \
 0      삼성전자      삼성물산   78676.0    2269950.0  149436938.0  16821275.0   
 1      삼성전자  삼성바이오에피스       1.0    2269950.0  149436938.0  16821275.0   
 2      삼성전자   삼성디스플레이  164404.0    2269950.0  149436938.0  16821275.0   
 3      삼성전자       세메스       5.0    2269950.0  149436938.0  16821275.0   
 4      삼성전자   삼성전자서비스  456109.0    2269950.0  149436938.0  16821275.0   
 ..      ...       ...       ...          ...          ...         ...   
 176  한국정보인증  키움인베스트먼트       0.0        142.0          NaN     38913.0   
 177  한국정보인증    키움저축은행      24.0        142.0          NaN     38913.0   
 178  한국정보인증  키움투자자산운용       0.0        142.0          NaN     38913.0   
 179  한국정보인증  키움예스저축은행       6.0        142.0          NaN     38913.0   
 180  한국정보인증      키움증권      15.0        142.0          NaN     38913.0   
 
            해외매출액        매출액총계  year2  year grpname2    매출회사id 매출사상/비  \
 0    153560595.0  170381870.0   2019

In [6]:
myApi.mergeTransId(2020)

=============== 삼성 ===============
[]
['생보부동산신탁', '네추럴나인']
   매출회사      매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액        해외매출액  \
0  삼성전자      삼성물산  110238      2422149  130563681.0  20300939  134471920.0   
1  삼성전자  삼성바이오에피스       4      2422149  130563681.0  20300939  134471920.0   
2  삼성전자   삼성디스플레이  279363      2422149  130563681.0  20300939  134471920.0   
3  삼성전자       세메스       7      2422149  130563681.0  20300939  134471920.0   
4  삼성전자   삼성전자서비스  453084      2422149  130563681.0  20300939  134471920.0   

         매출액총계  year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융       매출액Val  \
0  154772859.0   2020  2019       삼성  380725.0     상장       비금융  154772859.0   
1  154772859.0   2020  2019       삼성  380725.0     상장       비금융  154772859.0   
2  154772859.0   2020  2019       삼성  380725.0     상장       비금융  154772859.0   
3  154772859.0   2020  2019       삼성  380725.0     상장       비금융  154772859.0   
4  154772859.0   2020  2019       삼성  380725.0     상장       비금융  1547728

=============== 엘지 ===============
=============== 롯데 ===============
[]
['동교청기와피에프브이', '바이더웨이', '롯데피에스넷', '롯데첨단소재', '롯데로지스틱스', '현대정보기술', '디시네마오브코리아', '롯데카드', '이비카드', '경기스마트카드', '인천스마트카드', '롯데손해보험', '마이비', '부산하나로카드', '한페이시스', 'VietnamLotteriaCo.,Ltd(vietnam)', 'PT.LOTTERIAINDONESIA', 'JSCLotteRUS', 'PT.SJIndonesia', 'LotteDutyFreeGuam,LLC', 'LottePhuKhanhDutyFreeTradingCo.,Ltd', 'HAITHANH-LOTTECOMPANYLIMITED', 'LotteHotelGuamLLC', 'LotteHotelVietnamCompanyLimited', 'LotteDutyFreeKansaiCo.,Ltd', 'LotteDutyFreeJapanCo.,Ltd', 'LotteDutyFree(Thailand)Co.,Ltd', 'LotteHotelSaintPetersburg,LLC', 'LotteHotelAraiCo.,Ltd', 'LotteHotelHoldingsUSACo.,Ltd', 'LotteHotelNewYorkPalaceLLC', 'LotteHotelVladivostokLLC', 'LotteEuropeInvestmentB.V', '낙천(중국)식품유한공사', 'LOTTEQINGDAOFOODSCO.,LTD.,', '낙천(상해)식품유한공사', 'LotteConfectioneryRUSLLC', 'LotteIndiaCo.,Ltd.(제과)', 'ChocolaterieGuylianN.V', 'LotteKolson(Pvt.)Limited', 'LotteConfectionery(S.E.A.)Pte.Ltd.,', 'RakhatJSC', 'LotteInternationalAmericaCorp', 'Lotte

=============== 지에스 ===============
=============== 현대중공업 ===============
[]
['현대힘스', '현대중공업터보기계', 'HyundaiHeavyIndustriesBrasil-ManufacturingandTradingofConstructionEquipment', 'HyundaiArabiaLLC', '현대중공(중국)투자유한공사', 'HyundaiConstructionEquipmentEuropeN.V', 'HyundaiConstructionEquipmentAmericas,Inc', 'HyundaiConstructionEquipmentIndiaPrivateLtd', '상주현대액압기기유한공사', '현대융자조임유한공사', '현대강소공정기계유한공사', '연대현대빙윤중공유한공사', 'HyundaiHeavyIndustriesCo.Bulgaria', 'HyundaiPowerTransformersUSA,Inc', '현대중공(중국)전기유한공사', 'Hyundai-VietnamShipbuildingCo.,Ltd.', 'HyundaiOilSingaporePte.Ltd', 'HyundaiOilbank(SHANGHAI)CO.,LTD', 'HyundaiGlobalServiceEuropeB.V', 'HyundaiGlobalServiceAmericasCo.,Ltd', 'HyundaiGlobalServiceSingaporePte.Ltd', 'HYUNDAIENERGYSOLUTIONSAMERICAINC', 'HYUNDAIROBOTICS(SHANGHAI)CO.,LTD', 'PT.HyundaiConstructionEquipmentAsia']
     매출회사      매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액      해외매출액  \
0  한국조선해양    현대미포조선  154880       414937       1028.0  421747  2935082.0   
1  한국조선해양   현대삼호중공업  

=============== 두산 ===============
=============== 엘에스 ===============
[]
['거래업체수엘에스는주석참조', '쿠퍼스탠다드오토모티브앤인더스트리얼', '예스코서비스']
  매출회사      매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액  해외매출액    매출액총계  \
0  엘에스     엘에스전선  9849      33355.0         3342  83768.0   3342  87110.0   
1  엘에스   엘에스일렉트릭  6794      33355.0         3342  83768.0   3342  87110.0   
2  엘에스  엘에스니꼬동제련  8262      33355.0         3342  83768.0   3342  87110.0   
3  엘에스    엘에스엠트론  3180      33355.0         3342  83768.0   3342  87110.0   
4  엘에스        이원   246      33355.0         3342  83768.0   3342  87110.0   

   year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융   매출액Val      매입회사id  \
0   2020  2019      엘에스  380377.0     상장       비금융  87110.0  13017566.0   
1   2020  2019      엘에스  380377.0     상장       비금융  87110.0    388696.0   
2   2020  2019      엘에스  380377.0     상장       비금융  87110.0    109492.0   
3   2020  2019      엘에스  380377.0     상장       비금융  87110.0  13017572.0   
4   2020  2019      엘에스  380377.0     

[]
['KIARACAPITALⅡPTE.LTD', 'KISVietnamSecuritiesCorporation', 'KoreaInvestment&SecuritiesAsia,Ltd', 'KIMInvestmentFunds', 'POSCONPSNiobiumLLC', 'EQPCooperatiefU.A', '케이아이피플러스', '이큐그린제일차', '이큐그린제삼차', '드림라인쏠라파워1호']
       매출회사      매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액   해외매출액  \
0  한국투자금융지주    한국투자증권  2246        14000       3517.0  17137.0  3517.0   
1  한국투자금융지주  한국투자신탁운용   312        14000       3517.0  17137.0  3517.0   
2  한국투자금융지주   한국투자캐피탈  2554        14000       3517.0  17137.0  3517.0   
3  한국투자금융지주  한국투자저축은행  1320        14000       3517.0  17137.0  3517.0   
4  한국투자금융지주  한국투자파트너스  7566        14000       3517.0  17137.0  3517.0   

     매출액총계  year2  year grpname2   매출회사id 매출사상/비 매출사금융/비금융    매출액Val  \
0  20654.0   2020  2019   한국투자금융  97600.0     상장        금융  451215.0   
1  20654.0   2020  2019   한국투자금융  97600.0     상장        금융  451215.0   
2  20654.0   2020  2019   한국투자금융  97600.0     상장        금융  451215.0   
3  20654.0   2020  2019   한국투자금융  97600.0     상장     

=============== 케이씨씨 ===============
=============== 코오롱 ===============
[]
['코오롱화이버', '셀빅개발', '데크씨에스피']
  매출회사      매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)  국내매출액  해외매출액    매출액총계  \
0  코오롱  코오롱인더스트리  16654        43174        130.0  60332  130.0  60462.0   
1  코오롱   코오롱플라스틱   2751        43174        130.0  60332  130.0  60462.0   
2  코오롱   코오롱머티리얼   2553        43174        130.0  60332  130.0  60462.0   
3  코오롱    코오롱글로텍   3129        43174        130.0  60332  130.0  60462.0   
4  코오롱      그린나래      1        43174        130.0  60332  130.0  60462.0   

   year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융   매출액Val      매입회사id  \
0   2020  2019      코오롱  320234.0     상장       비금융  60462.0  16003302.0   
1   2020  2019      코오롱  320234.0     상장       비금융  60462.0    825841.0   
2   2020  2019      코오롱  320234.0     상장       비금융  60462.0  13004456.0   
3   2020  2019      코오롱  320234.0     상장       비금융  60462.0    325732.0   
4   2020  2019      코오롱  320234.0     상장       비금융  60462.0    

[]
['소호브릭스', '넥슨레드', 'ATMediaInvestor,LLC', 'NEXONCo.,Ltd', 'NEXONAmerica,Inc', 'NEXONMInc', 'NEXON(Thailand)Co.,Ltd', 'NEXONTAIWANLIMITED', 'STOKKEAS', '불리언게임즈', 'BigHugeGames,Inc']
   매출회사    매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)  국내매출액   해외매출액    매출액총계  \
0  엔엑스씨  아퀴스코리아    65         4836       7766.0  13737  8548.0  22285.0   
1  엔엑스씨    가승개발     7         4836       7766.0  13737  8548.0  22285.0   
2  엔엑스씨    가승개발     7         4836       7766.0  13737  8548.0  22285.0   
3  엔엑스씨   넥슨코리아  2774         4836       7766.0  13737  8548.0  22285.0   
4  엔엑스씨     네오플  1568         4836       7766.0  13737  8548.0  22285.0   

   year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융   매출액Val        매입회사id  \
0   2020  2019       넥슨  799173.0    비상장       비금융  22285.0  1.809424e+07   
1   2020  2019       넥슨  799173.0    비상장       비금융  22285.0  1.802408e+09   
2   2020  2019       넥슨  799173.0    비상장       비금융  22285.0  1.802408e+09   
3   2020  2019       넥슨  799173.0    비상장       비금융  22285.

[]
['동원냉장', 'StarKistCo', 'SCASA', 'TTP', 'DWGlobalInc', 'DongwonJapanCo.,Ltd', '동원식품(상해)유한공사', 'DongwonF&BVietnamCo.,Ltd', 'StarKistSamoaCo', 'GalapescaS.A', 'CAPSEN', 'KKT', 'TALOFA', 'MVP', '위해삼조유한공사', '상해동부', 'DU', 'DV']
       매출회사    매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)  국내매출액   해외매출액    매출액총계  \
0  동원엔터프라이즈  동원에프앤비  12252        42154       1949.0  43943  1949.0  45892.0   
1  동원엔터프라이즈    동원산업   4892        42154       1949.0  43943  1949.0  45892.0   
2  동원엔터프라이즈  동원시스템즈   4301        42154       1949.0  43943  1949.0  45892.0   
3  동원엔터프라이즈  동원건설산업   1958        42154       1949.0  43943  1949.0  45892.0   
4  동원엔터프라이즈    동원팜스   2018        42154       1949.0  43943  1949.0  45892.0   

   year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융   매출액Val      매입회사id  \
0   2020  2019       동원  164326.0    비상장       비금융  75885.0    152859.0   
1   2020  2019       동원  164326.0    비상장       비금융  75885.0    100030.0   
2   2020  2019       동원  164326.0    비상장       비금융  75885.0    388742

     매출회사     매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액      해외매출액  \
0  금호석유화학  금호피앤비화학  153335       189833      20242.0  1220149  2478974.0   
1  금호석유화학  금호미쓰이화학    8681       189833      20242.0  1220149  2478974.0   
2  금호석유화학  금호미쓰이화학    8681       189833      20242.0  1220149  2478974.0   
3  금호석유화학    금호폴리켐   23178       189833      20242.0  1220149  2478974.0   
4  금호석유화학   금호개발상사     185       189833      20242.0  1220149  2478974.0   

       매출액총계  year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융     매출액Val  \
0  3699123.0   2020  2019   금호석유화학  351350.0     상장       비금융  3699123.0   
1  3699123.0   2020  2019   금호석유화학  351350.0     상장       비금융  3699123.0   
2  3699123.0   2020  2019   금호석유화학  351350.0     상장       비금융  3699123.0   
3  3699123.0   2020  2019   금호석유화학  351350.0     상장       비금융  3699123.0   
4  3699123.0   2020  2019   금호석유화학  351350.0     상장       비금융  3699123.0   

     매입회사id 매입사상/비 매입사금융/비금융  
0  145509.0    비상장       비금융  
1  356468.0    비상장      

(      매출회사      매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)       국내매출액  \
 0     삼성전자      삼성물산  110238.0    2422149.0  130563681.0  20300939.0   
 1     삼성전자  삼성바이오에피스       4.0    2422149.0  130563681.0  20300939.0   
 2     삼성전자   삼성디스플레이  279363.0    2422149.0  130563681.0  20300939.0   
 3     삼성전자       세메스       7.0    2422149.0  130563681.0  20300939.0   
 4     삼성전자   삼성전자서비스  453084.0    2422149.0  130563681.0  20300939.0   
 ..     ...       ...       ...          ...          ...         ...   
 45  삼양에프앤비     삼양패키징       7.0        392.0          NaN     12987.0   
 46  삼양에프앤비    삼양바이오팜      52.0        392.0          NaN     12987.0   
 47  삼양에프앤비  삼양데이타시스템      15.0        392.0          NaN     12987.0   
 48     메디켐    삼양바이오팜    2319.0       2484.0          NaN      3903.0   
 49     메디켐      삼양화성     165.0       2484.0          NaN      3903.0   
 
           해외매출액        매출액총계  year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융  \
 0   134471920.0  154772859.0   2020  20

In [ ]:
myApi.mergeTransId(2021)

=============== 삼성 ===============
[]
[]
   매출회사      매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액        해외매출액  \
0  삼성전자      삼성물산    86705      3142513  141902290.0  19833135  146478056.0   
1  삼성전자  삼성바이오에피스        0      3142513  141902290.0  19833135  146478056.0   
2  삼성전자   삼성디스플레이   196823      3142513  141902290.0  19833135  146478056.0   
3  삼성전자   삼성전자서비스   465778      3142513  141902290.0  19833135  146478056.0   
4  삼성전자    삼성전자판매  2225711      3142513  141902290.0  19833135  146478056.0   

         매출액총계  year2  year grpname2  매출회사id 매출사상/비 매출사금융/비금융        매출액Val  \
0  166311191.0   2021  2020       삼성  380725     상장       비금융  1.660000e+14   
1  166311191.0   2021  2020       삼성  380725     상장       비금융  1.660000e+14   
2  166311191.0   2021  2020       삼성  380725     상장       비금융  1.660000e+14   
3  166311191.0   2021  2020       삼성  380725     상장       비금융  1.660000e+14   
4  166311191.0   2021  2020       삼성  380725     상장       비금융  1.660000e+14   

     매입회사

=============== 롯데 ===============
=============== 한화 ===============
['한화갤러리아']
['한화갤러리아', '한화금융에셋', 'HanwhaMiningServicesAustraliaPty.,Ltd', 'HanwhaChemical(Ningbo)Co.,Ltd', 'HanwhaChemical(Thailand)Co.,Ltd', 'HanwhaQCELLSGmbh', 'HanwhaQCELLSMalaysiaSdn.Bhd', 'PT.HanwhaMiningServicesIndonesia', 'S&PWorldNetworksDMCC', 'HanwhaAZDEL,Inc', 'HanwhaAdvancedMaterialsBeijingCo.,Ltd', 'HanwhaAdvancedMaterialsEurope,s.r.o', 'HanwhaAdvancedMaterialsShanghaiCo.,Ltd', 'HanwhaHongKongCo.,Ltd', 'HanwhaInternationalLLC', 'HanwhaQCELLSJapanCo.,Ltd', 'HanwhaQCELLSChileSpA', 'HanwhaEuropeGmbH', 'HanwhaInternational(S)PteLtd', 'HanwhaAdvancedMaterialsAmericaLLC', 'HanwhaQCELLS(Qidong)Co.,Ltd', 'HanwhaQCELLSUSA,INC', 'HanwhaQCELLSEPCUSA,LLC', 'HanwhaMiningServicesChileSpA', 'HanwhaAdvancedMaterialsGermanyGmbH', 'HanwhaTechwinMiddleEastFZE', 'HanwhaTechwinSecurityVietnam', 'HanwhaQCELLSAmericasHoldings.Corp', '라온태양광', '마루태양광', '미르태양광', '소수태양광', '수리태양광', '아란태양광']
  매출회사        매입회사      매출액  국내계열사계(매출액)  

=============== 씨제이 ===============
=============== 한진 ===============
[]
['제동레저', 'HanjinInternationalCorp', "HanjinInt'lJapanCo.,Ltd", 'TotalAviationServiceLLC', 'HanjinIntermodalAmericaInc', 'QINGDAOHANJINLUHAIINTERNATIONALLOGISTICSCO.,LTD', 'HanjinGlobalLogistics(HongKong)Ltd', 'HANJINGLOBALLOGISTICS(SHANGHAI)LIMITED', 'HanjinGlobalLogistics(Dalian)Ltd', 'HANJINGLOBALLOGISTICS(SHENZHEN)LIMITED', 'HANJINGLOBALLOGISTICSEUROPEs.r.o', 'HANJINGLOBALLOGISTICSMYANMARCO.,LTD', 'HANJINGLOBALLOGISTICS(VIETNAM)LIMITED', 'GlobalLogisticsSystemAsiaPacificCo.,Ltd', "WACINT'LLOGISTICS(HK)LIMITED", "WACINT'LLOGISTICS(QINGDAO)CONPANYLIMITED"]
   매출회사     매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액      해외매출액  \
0  대한항공      한진칼   1158.0      65681.0        318.0  736452  6668554.0   
1  대한항공       한진    709.0      65681.0        318.0  736452  6668554.0   
2  대한항공      진에어  56518.0      65681.0        318.0  736452  6668554.0   
3  대한항공     정석기업    141.0      65681.0        318.0  736452  66685

=============== 미래에셋 ===============
=============== 현대백화점 ===============
[]
['매그놀리아코리아', '한섬파리', '한섬(상해)상무유한공사', 'HYUNDAILIVARTVINA', 'LIVARTCANADA', '현대휘상무역(상해)유한공사', 'HYUNDAIBIOLANDJIANGSUCO.LTDSKBIOLANDHAIMENCO.LTD)', '타이포스', '한우공정기계(상해)유한공사', 'EVERDIGMMONGOLIALLC', 'HyundaiL&CCanadaInc', 'HyundaiL&C(Shanghai)Co.,LTD', 'HyundaiL&CUSA,LLC', 'HyundaiL&CEuropeGmbH', 'AmericanSurfacingMaterials,LLC', 'AUST-RAILIANSHOPPINGNETWORKPTYLTD', 'HIGHShoppingCo.,Ltd']
    매출회사       매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액  해외매출액  매출액총계  \
0  현대백화점  씨엔에스푸드시스템     26.0       188596          NaN  1245063    NaN    NaN   
1  현대백화점         한섬  51565.0       188596          NaN  1245063    NaN    NaN   
2  현대백화점     한섬라이프앤      3.0       188596          NaN  1245063    NaN    NaN   
3  현대백화점     현대그린푸드  39336.0       188596          NaN  1245063    NaN    NaN   
4  현대백화점     현대드림투어    243.0       188596          NaN  1245063    NaN    NaN   

   year2  year grpname2  매출회사id 매출사상/비 매출사금융/비금융